In [ ]:
from google.colab import auth
auth.authenticate_user()  # 先做一次账号授权

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

%cd drive/MyDrive/ESE5380TimeSeries

/content/drive/MyDrive/ESE5380TimeSeries


#### Download package

In [ ]:
# 1) 升级 pip（用当前 Python 的 pip）
!pip install -U pip

# 2) 安装 PyTorch 2.4.0 + cu118（三件套）
# !pip install \
#   "torch==2.4.0+cu118" \
#   "torchvision==0.19.0+cu118" \
#   "torchaudio==2.4.0+cu118" \
#   --extra-index-url https://download.pytorch.org/whl/cu118

# 3) 安装 xarray + zarr + numcodecs + gcsfs
#    （numcodecs 是刚才缺的那个 _shuffle 扩展）
!pip install \
  "numpy" \
  "pandas" \
  "xarray>=2024.6.0" \
  "zarr>=2.18.0" \
  "numcodecs>=0.13.1" \
  "gcsfs>=2024.6.0"

!python -m pip install "dask[complete]"
!pip install zarr numcodecs gcsfs fsspec



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 99.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [zarr]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.3 MB/s  0:00:00


#### Download Data

In [ ]:
import xarray as xr
import numpy as np
import os, time

print("xarray version:", xr.__version__)

# gcs_path_6h = (
#     "gs://weatherbench2/datasets/era5/"
#     "1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr"
# )

gcs_path_1h = (
    "gs://weatherbench2/datasets/era5/"
    "1959-2023_01_10-1h-240x121_equiangular_with_poles_conservative.zarr" # 1.5 deg resolution
)

# 1959-2022-1h-240x121_equiangular_with_poles_conservative.zarr --> 1.5 deg resolution
# weatherbench2/datasets/era5/1959-2023_01_10-1h-240x121_equiangular_with_poles_conservative.zarr


# print(f"正在打开 GCS 路径 (1.5-deg, 6-hourly): {gcs_path_6h}")

# ds_6h = xr.open_dataset(
#     gcs_path_6h,
#     engine="zarr",
#     chunks={},
#     storage_options={"token": "anon"},
# )
# print("6-hourly 数据集元数据加载成功。")


### valid
ds_1h = xr.open_dataset(
    gcs_path_1h,
    engine="zarr",
    chunks={},
    storage_options={"token": "anon"},
)
print("1-hourly 数据集元数据加载成功。")

# data_train = ds_6h.sel(time=slice("2019-01-01", "2021-12-31"))

# data_2022 = ds_6h.sel(time=slice("2022-01-01", "2022-12-31")) # test dataset



data_train = ds_1h.sel(time=slice("2019-01-01", "2020-12-31"))
data_val   = ds_1h.sel(time=slice("2021-01-01", "2021-12-31"))
data_test  = ds_1h.sel(time=slice("2022-01-01", "2022-12-31"))




variables = [
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "mean_sea_level_pressure",
    "total_precipitation"
]
data_vars = data_train[variables]



# # 裁剪纬度: 121 -> 112 (H)
# data_final_lazy = data_vars.isel(latitude=slice(0, 112))

# H:121 * W:240

data_final_lazy = data_vars
print(data_final_lazy)
print(f"T={len(data_final_lazy.time)}, "
      f"H={len(data_final_lazy.latitude)}, "
      f"W={len(data_final_lazy.longitude)}, "
      f"C={len(data_final_lazy.data_vars)}")

# array: dims: (variable, time, latitude, longitude)
data_array = data_final_lazy.to_array(dim="variable")
# convert to (T, H, W, C)
data_np = data_array.transpose("time", "latitude", "longitude", "variable").values.astype(np.float32)

T_all, H, W, C = data_np.shape
print(f"all_data shape: {data_np.shape}")

xarray version: 2025.12.0
1-hourly 数据集元数据加载成功。
<xarray.Dataset> Size: 10GB
Dimensions:                  (time: 17544, longitude: 240, latitude: 121)
Coordinates:
  * time                     (time) datetime64[ns] 140kB 2019-01-01 ... 2020-...
  * longitude                (longitude) float64 2kB 0.0 1.5 3.0 ... 357.0 358.5
  * latitude                 (latitude) float64 968B -90.0 -88.5 ... 88.5 90.0
Data variables:
    2m_temperature           (time, longitude, latitude) float32 2GB dask.array<chunksize=(24, 240, 121), meta=np.ndarray>
    10m_u_component_of_wind  (time, longitude, latitude) float32 2GB dask.array<chunksize=(24, 240, 121), meta=np.ndarray>
    10m_v_component_of_wind  (time, longitude, latitude) float32 2GB dask.array<chunksize=(24, 240, 121), meta=np.ndarray>
    mean_sea_level_pressure  (time, longitude, latitude) float32 2GB dask.array<chunksize=(24, 240, 121), meta=np.ndarray>
    total_precipitation      (time, longitude, latitude) float32 2GB dask.array<chunksize

#### Dataloader(Old Version)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

variables = [
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "mean_sea_level_pressure",
    "total_precipitation",
]

class ERA5XYDataset(Dataset):
    def __init__(self, data_np: np.ndarray, variables: list[str],
                 sample_step: int = 6,
                 x_len: int = 8, y_len: int = 4,
                 stride: int = 1,
                 normalize: bool = True):
        """
        Args:
            data_np: (T, H, W, C)
            variables: 变量名
            sample_step: 6h 聚合步长
        """
        self.variables = variables
        self.x_len = x_len
        self.y_len = y_len
        self.normalize = normalize

        # 找 total_precipitation 通道
        tp_idx = variables.index("total_precipitation") if "total_precipitation" in variables else None

        # --- 1️⃣ 构建 6 小时步数据 ---
        data_list = []
        for c in range(data_np.shape[-1]):
            if c == tp_idx:
                # 降水 → 过去6小时累计
                summed = np.add.reduceat(data_np[..., c], np.arange(0, data_np.shape[0], sample_step), axis=0)
                data_list.append(summed[..., None])
            else:
                # 其他变量 → 每6小时取当前值
                sampled = data_np[sample_step-1::sample_step, :, :, c]
                data_list.append(sampled[..., None])

        data_6h = np.concatenate(data_list, axis=-1)  # (T6,H,W,C)
        self.data = data_6h
        T, H, W, C = data_6h.shape

        # --- 2️⃣ 构造滑动窗口索引 ---
        Xs, Ys = [], []
        for start in range(0, T - (x_len + y_len), stride):
            x_idx = np.arange(start, start + x_len)
            y_idx = np.arange(start + x_len, start + x_len + y_len)
            Xs.append(x_idx)
            Ys.append(y_idx)
        self.indices = list(zip(Xs, Ys))

        # --- 3️⃣ 均值方差（只用于 normalization） ---
        mean = data_6h.mean(axis=(0, 1, 2), keepdims=True)
        std = data_6h.std(axis=(0, 1, 2), keepdims=True) + 1e-6
        self.mean = torch.from_numpy(mean).permute(3, 0, 1, 2).float()
        self.std = torch.from_numpy(std).permute(3, 0, 1, 2).float()

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        x_idx, y_idx = self.indices[idx]
        x = self.data[x_idx]  # (Tin, H, W, C)
        y = self.data[y_idx]  # (Tout, H, W, C)

        # 转换为 torch tensor
        x = torch.from_numpy(x).permute(0, 3, 1, 2).float()  # (Tin, C, H, W)
        y = torch.from_numpy(y).permute(0, 3, 1, 2).float()  # (Tout, C, H, W)

        if self.normalize:
            mean = self.mean.squeeze(1)  # (C,1,1)
            std = self.std.squeeze(1)    # (C,1,1)
            x = (x - mean) / std         # 广播: Tin 与 (C,1,1)
            y = (y - mean) / std
        return x, y


def xr_to_np(ds):
    arr = ds[variables].to_array(dim="variable")
    return arr.transpose("time", "latitude", "longitude", "variable").values.astype(np.float32)

np_train = xr_to_np(data_train)
np_val   = xr_to_np(data_val)

# ===== 假设你已经有 np_train, np_val =====
# 比如你前面用 xarray 转好了：
# import xarray as xr
# def xr_to_np(ds):
#     arr = ds[variables].to_array(dim="variable")
#     return arr.transpose("time", "latitude", "longitude", "variable").values.astype(np.float32)
# np_train = xr_to_np(data_train)
# np_val   = xr_to_np(data_val)

ds_train = ERA5XYDataset(np_train, variables)
ds_val   = ERA5XYDataset(np_val, variables)

train_loader = DataLoader(ds_train, batch_size=2, shuffle=True,  num_workers=0, pin_memory=False)
val_loader   = DataLoader(ds_val,   batch_size=2, shuffle=False, num_workers=0, pin_memory=False)

print(f"Train samples: {len(ds_train)} | Val samples: {len(ds_val)}")
x_in, y_out = next(iter(train_loader))
print("x_in:", x_in.shape)
print("y_out:", y_out.shape)


Train samples: 2912 | Val samples: 1448
x_in: torch.Size([2, 8, 5, 121, 240])
y_out: torch.Size([2, 4, 5, 121, 240])


#### Dataloader(New version)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

variables = [
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "mean_sea_level_pressure",
    "total_precipitation",
]

class ERA5XYDataset(Dataset):
    def __init__(
        self,
        data_np: np.ndarray,          # (T, H, W, C)
        variables: list[str],
        sample_step: int = 6,
        x_len: int = 8,
        y_len: int = 4,
        stride: int = 1,
        normalize: bool = True,
        mean: torch.Tensor | None = None,   # 允许外部传入 mean/std (C,1,1,1) or (1,1,1,C)
        std:  torch.Tensor | None = None,
    ):
        """
        Args:
            data_np: (T, H, W, C)
            variables: 变量名
            sample_step: 例如 6 表示从 1h 分辨率聚合到 6h
            x_len: 输入序列长度（时间步数）
            y_len: 预测序列长度（时间步数）
            stride: 滑动窗口步长
            normalize: 是否返回标准化后的 x,y
            mean/std: 若不为 None，则使用给定的 mean/std 进行归一化
        """
        self.variables = variables
        self.x_len = x_len
        self.y_len = y_len
        self.normalize = normalize

        # -------- 1️⃣ 6h 聚合 --------
        tp_idx = variables.index("total_precipitation") if "total_precipitation" in variables else None

        data_list = []
        T_raw = data_np.shape[0]
        for c in range(data_np.shape[-1]):
            if c == tp_idx:
                # total_precipitation: 过去 6 小时累计
                summed = np.add.reduceat(
                    data_np[..., c],
                    np.arange(0, T_raw, sample_step),
                    axis=0
                )  # (T6, H, W)
                data_list.append(summed[..., None])  # (T6,H,W,1)
            else:
                # 其他变量: 每 6 小时取一个 instantaneous 值（比如 5,11,17,...）
                sampled = data_np[sample_step-1::sample_step, :, :, c]  # (T6,H,W)
                data_list.append(sampled[..., None])

        data_6h = np.concatenate(data_list, axis=-1)  # (T6, H, W, C)
        self.data = data_6h
        T, H, W, C = data_6h.shape

        # -------- 2️⃣ 滑动窗口索引（修复 off-by-one） --------
        Xs, Ys = [], []
        max_start = T - (x_len + y_len)
        for start in range(0, max_start + 1, stride):
            x_idx = np.arange(start, start + x_len)
            y_idx = np.arange(start + x_len, start + x_len + y_len)
            Xs.append(x_idx)
            Ys.append(y_idx)
        self.indices = list(zip(Xs, Ys))

        # -------- 3️⃣ 计算或接收 mean/std --------
        if mean is None or std is None:
            # 按 (T,H,W) 维度求各通道统计量
            mean_np = data_6h.mean(axis=(0, 1, 2), keepdims=True)  # (1,1,1,C)
            std_np  = data_6h.std(axis=(0, 1, 2), keepdims=True) + 1e-6
            mean_t  = torch.from_numpy(mean_np).permute(3, 0, 1, 2).float()  # (C,1,1,1)
            std_t   = torch.from_numpy(std_np ).permute(3, 0, 1, 2).float()  # (C,1,1,1)
        else:
            # 允许传入 (C,1,1,1) 或 (1,C,1,1) 等，统一成 (C,1,1,1)
            mean_t = mean
            std_t  = std
            if mean_t.ndim == 4 and mean_t.shape[0] == 1:
                # (1,C,1,1) -> (C,1,1,1)
                mean_t = mean_t.permute(1,0,2,3).contiguous()
            if std_t.ndim == 4 and std_t.shape[0] == 1:
                std_t = std_t.permute(1,0,2,3).contiguous()
            mean_t = mean_t.float()
            std_t  = std_t.float()

        self.mean = mean_t  # (C,1,1,1)
        self.std  = std_t   # (C,1,1,1)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        x_idx, y_idx = self.indices[idx]
        x = self.data[x_idx]  # (Tin, H, W, C)
        y = self.data[y_idx]  # (Tout, H, W, C)

        # -> torch & 调整维度为 (T,C,H,W)
        x = torch.from_numpy(x).permute(0, 3, 1, 2).float()
        y = torch.from_numpy(y).permute(0, 3, 1, 2).float()

        if self.normalize:
            # self.mean/self.std: (C,1,1,1)
            mean = self.mean.squeeze(1)  # (C,1,1)
            std  = self.std.squeeze(1)   # (C,1,1)
            x = (x - mean) / std
            y = (y - mean) / std

        return x, y


In [ ]:
import xarray as xr

def xr_to_np(ds):
    # ds: xarray.Dataset，包含 variables 中的变量
    arr = ds[variables].to_array(dim="variable") \
                       .transpose("time", "latitude", "longitude", "variable")
    return arr.values.astype(np.float32)  # (T,H,W,C)

# 假设你已经有 data_train, data_val 两个 xarray.Dataset
np_train = xr_to_np(data_train)
np_val   = xr_to_np(data_val)

# 1) 先构建 train dataset，让它自己算 mean/std
ds_train = ERA5XYDataset(
    np_train,
    variables,
    sample_step=6,
    x_len=8,
    y_len=4,
    stride=1,
    normalize=True,
)

# 2) val 复用 train 的 mean/std（保证归一化一致）
ds_val = ERA5XYDataset(
    np_val,
    variables,
    sample_step=6,
    x_len=8,
    y_len=4,
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
)

# 3) DataLoader
train_loader = DataLoader(ds_train, batch_size=2, shuffle=True,  num_workers=0, pin_memory=False)
val_loader   = DataLoader(ds_val,   batch_size=2, shuffle=False, num_workers=0, pin_memory=False)

print(f"Train samples: {len(ds_train)} | Val samples: {len(ds_val)}")
x_in, y_out = next(iter(train_loader))
print("x_in:", x_in.shape)   # (B, Tin, C, H, W)
print("y_out:", y_out.shape) # (B, Tout, C, H, W)


Train samples: 2913 | Val samples: 1449
x_in: torch.Size([2, 8, 5, 121, 240])
y_out: torch.Size([2, 4, 5, 121, 240])


#### Dataloader(6h New version)

In [ ]:
import xarray as xr

print("xarray version:", xr.__version__)

# 6-hourly ERA5 (WeatherBench2 提供的 1.5° 数据)
gcs_path_6h = (
    "gs://weatherbench2/datasets/era5/"
    "1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr"
)

print(f"正在打开 GCS 路径 (1.5-deg, 6-hourly): {gcs_path_6h}")

ds_6h = xr.open_dataset(
    gcs_path_6h,
    engine="zarr",
    chunks={},
    storage_options={"token": "anon"},
)

print("6-hourly 数据集元数据加载成功。")
print(ds_6h)


xarray version: 2025.12.0
正在打开 GCS 路径 (1.5-deg, 6-hourly): gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr
6-hourly 数据集元数据加载成功。
<xarray.Dataset> Size: 2TB
Dimensions:                                           (time: 93544,
                                                       longitude: 240,
                                                       latitude: 121, level: 13)
Coordinates:
  * time                                              (time) datetime64[ns] 748kB ...
  * longitude                                         (longitude) float64 2kB ...
  * latitude                                          (latitude) float64 968B ...
  * level                                             (level) int64 104B 50 ....
Data variables: (12/62)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 11GB dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    10m_v_component_of_wind                           (t

In [ ]:

data_train = ds_6h.sel(time=slice("2019-01-01", "2020-12-31"))
data_val   = ds_6h.sel(time=slice("2021-01-01", "2021-12-31"))
data_test  = ds_6h.sel(time=slice("2022-01-01", "2022-12-31"))


In [ ]:
variables = [
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "mean_sea_level_pressure",
    "total_precipitation_6hr",
]

def xr_to_np_6h(ds, variables):
    # ds: xarray.Dataset
    arr = ds[variables].to_array(dim="variable") \
                       .transpose("time", "latitude", "longitude", "variable")
    return arr.values.astype(np.float32)  # (T,H,W,C)

np_train = xr_to_np_6h(data_train, variables)
np_val   = xr_to_np_6h(data_val,   variables)
np_test  = xr_to_np_6h(data_test,  variables)

print("np_train shape:", np_train.shape)
print("np_val shape:",   np_val.shape)
print("np_test shape:",  np_test.shape)


np_train shape: (2924, 121, 240, 5)
np_val shape: (1460, 121, 240, 5)
np_test shape: (1460, 121, 240, 5)


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# 变量名
variables = [
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "mean_sea_level_pressure",
    "total_precipitation_6hr",
]


class ERA5XYDataset6h(Dataset):
    """
    专门用于已经是 6 小时间隔的数据：
    - 不再做 1h -> 6h 聚合
    - 只做时间滑窗 + 归一化
    - 降水通道可以选择 m->mm、log1p 变换
    """
    def __init__(
        self,
        data_np: np.ndarray,
        variables: list[str],
        x_len: int = 8,
        y_len: int = 4,
        stride: int = 1,
        normalize: bool = True,
        mean: torch.Tensor | None = None,   # (1,C,1,1) 或兼容形状
        std:  torch.Tensor | None = None,
        precip_to_mm: bool = True,          # 是否把降水从 m 转成 mm
        precip_log1p: bool = False,         # 是否对降水做 log1p 变换
    ):
        """
        Args:
            data_np: (T, H, W, C) 6-hourly data
            variables: 变量名列表
            x_len: 输入序列长度（时间步数）
            y_len: 预测序列长度（时间步数）
            stride: 滑动窗口步长
            normalize: 是否做标准化
            mean/std: 若不为 None，则复用给定统计量（保证 train/val 一致）
            precip_to_mm: 将降水从 m 转为 mm（乘 1000）
            precip_log1p: 对降水做 log1p 变换（在乘 mm 之后）
        """
        assert data_np.ndim == 4, f"data_np shape must be (T,H,W,C), got {data_np.shape}"
        self.variables = variables
        self.x_len = x_len
        self.y_len = y_len
        self.normalize = normalize

        T, H, W, C = data_np.shape

        # ---------- 找到降水通道索引 ----------
        precip_candidates = [
            "total_precipitation",
            "total_precipitation_6hr",
            "total_precipitation_6h",
        ]
        tp_idx = None
        for name in precip_candidates:
            if name in variables:
                tp_idx = variables.index(name)
                break
        self.tp_idx = tp_idx

        # ----------  对降水做可选变换 ----------
        data_proc = data_np.astype(np.float32).copy()
        if tp_idx is not None:
            if precip_to_mm:
                data_proc[..., tp_idx] *= 1000.0  # m -> mm

            if precip_log1p:
                # log(1 + mm)，保证非负安全
                data_proc[..., tp_idx] = np.log1p(np.clip(data_proc[..., tp_idx], a_min=0.0, a_max=None))

        self.data = data_proc  # (T,H,W,C)

        # ---------- 生成滑动窗口索引 ----------
        max_start = T - (x_len + y_len)
        if max_start < 0:
            raise ValueError(
                f"Time dimension T={T} is too short for x_len={x_len}, "
                f"y_len={y_len}"
            )

        Xs, Ys = [], []
        for start in range(0, max_start + 1, stride):
            x_idx = np.arange(start, start + x_len)
            y_idx = np.arange(start + x_len, start + x_len + y_len)
            Xs.append(x_idx)
            Ys.append(y_idx)
        self.indices = list(zip(Xs, Ys))

        # ---------- 计算或接收 mean/std ----------
        if mean is None or std is None:
            # 按 (T,H,W) 维度求各通道统计量
            mean_np = self.data.mean(axis=(0, 1, 2), keepdims=True)  # (1,1,1,C)
            std_np  = self.data.std(axis=(0, 1, 2), keepdims=True) + 1e-6

            mean_t = torch.from_numpy(mean_np).permute(0, 3, 1, 2).float()  # (1,C,1,1)
            std_t  = torch.from_numpy(std_np ).permute(0, 3, 1, 2).float()  # (1,C,1,1)
        else:
            # 统一成 (1,C,1,1)
            mean_t = mean
            std_t  = std

            if mean_t.ndim == 1:
                # (C,) -> (1,C,1,1)
                mean_t = mean_t.view(1, -1, 1, 1)
            elif mean_t.ndim == 3 and mean_t.shape[0] == len(variables):
                # (C,H,W) -> (1,C,H,W)
                mean_t = mean_t.unsqueeze(0)
            elif mean_t.ndim == 4 and mean_t.shape[0] != 1:
                # (C,1,1,1) -> (1,C,1,1)
                mean_t = mean_t.permute(1, 0, 2, 3).contiguous()

            if std_t.ndim == 1:
                std_t = std_t.view(1, -1, 1, 1)
            elif std_t.ndim == 3 and std_t.shape[0] == len(variables):
                std_t = std_t.unsqueeze(0)
            elif std_t.ndim == 4 and std_t.shape[0] != 1:
                std_t = std_t.permute(1, 0, 2, 3).contiguous()

            mean_t = mean_t.float()
            std_t  = std_t.float()

        self.mean = mean_t  # (1,C,1,1)
        self.std  = std_t   # (1,C,1,1)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx: int):
        x_idx, y_idx = self.indices[idx]
        x = self.data[x_idx]  # (Tin, H, W, C)
        y = self.data[y_idx]  # (Tout, H, W, C)

        # ->  (T,C,H,W)
        x = torch.from_numpy(x).permute(0, 3, 1, 2).float()
        y = torch.from_numpy(y).permute(0, 3, 1, 2).float()

        if self.normalize:
            x = (x - self.mean) / self.std
            y = (y - self.mean) / self.std

        return x, y


In [ ]:
from torch.utils.data import DataLoader

# 1) train：自动计算 mean/std
ds_train = ERA5XYDataset6h(
    np_train,
    variables,
    x_len=8,
    y_len=4,
    stride=1,
    normalize=True,
    precip_to_mm=True,   # m -> mm
    precip_log1p=True,
)

# 2) val / test 复用 train 的 mean/std
ds_val = ERA5XYDataset6h(
    np_val,
    variables,
    x_len=8,
    y_len=4,
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
    precip_to_mm=True,
    precip_log1p=True,
)

ds_test = ERA5XYDataset6h(
    np_test,
    variables,
    x_len=8,
    y_len=4,
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
    precip_to_mm=True,
    precip_log1p=True,
)


batch_size = 8

train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True,  num_workers=0)
val_loader   = DataLoader(ds_val,   batch_size=batch_size, shuffle=False, num_workers=0)
test_loader  = DataLoader(ds_test,  batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train samples: {len(ds_train)} | Val: {len(ds_val)} | Test: {len(ds_test)}")

x_in, y_out = next(iter(train_loader))
print("x_in:", x_in.shape)   # (B, Tin, C, H, W)
print("y_out:", y_out.shape) # (B, Tout, C, H, W)


Train samples: 2913 | Val: 1449 | Test: 1449
x_in: torch.Size([8, 8, 5, 121, 240])
y_out: torch.Size([8, 4, 5, 121, 240])


#### model

In [ ]:

import os
import json
import csv
from pathlib import Path
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List, Tuple, Optional, Union

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ============================================================
# =============== 0) 变量名（channels） =======================
# ============================================================

VARIABLES = variables

# ============================================================
# =============== 1) 配置（Config） ===========================
# ============================================================
@dataclass
class TrainConfig:
    # ---- 数据 ----
    # data_dir: str = "./"
    # z_name: str = "Z_2020_seq24.npy"
    # meta_name: str = "Z_2020_seq24_meta.npz"
    # stats_name: str = "Z_2020_norm_stats_2020.npz"

    # ---- 训练 ----
    batch_size: int = 2
    lr: float = 1e-3
    max_epochs: int = 50
    num_workers: int = 0
    grad_clip: float = 1.0

    # ---- Teacher Forcing ----
    enable_tf_ratio: bool = True
    teacher_forcing_start: float = 0.7
    teacher_forcing_end: float = 0.2

    # ---- 架构 ----
    hidden_dim: int = 32
    enc_cell_type: str = "convlstm"   # convrnn | convgru | convlstm
    dec_cell_type: str = "convlstm"   # convrnn | convgru | convlstm
    n_enc_layers: int = 1
    n_dec_layers: int = 1
    kernel_size: int = 3
    layernorm: bool = True
    peephole: bool = False            # 仅对 LSTM 有效
    unet_head: bool = False

    # ---- 可视化 ----
    out_root: str = "./runs"          # 根目录
    viz_every_n_epochs: int = 10       # 每隔 n 个 epoch 可视化一次（train + eval）
    viz_use_last_t: bool = True        # True: 用最后一帧；False: 随机一帧
    viz_channel: Optional[int] = None  # None: 随机通道；非 None：指定通道
    save_all_channels_png: bool = False # True：画所有通道；False：只画一个通道

# ============================================================
# =============== 2) 数据（Dataset） =========================
# ============================================================
class ERA5ZInOutDataset(Dataset):
    """
    Z 文件形状: (N, T, C, H, W)
    返回: (x_in, y_out) = (前 2 天, 后 1 天)  —— 以 meta 的 steps_per_day 为准
    """
    def __init__(self, data_dir, z_name, meta_name, stats_name, normalize=True, dtype=torch.float32):
        data_dir = Path(data_dir)
        self.Z_path = data_dir / z_name
        self.meta_path = data_dir / meta_name
        self.stats_path = data_dir / stats_name
        self.normalize = normalize
        self.dtype = dtype

        self.Z = np.load(self.Z_path, mmap_mode="r")
        self.N, self.T, self.C, self.H, self.W = self.Z.shape

        meta = np.load(self.meta_path)
        self.window_len = int(meta["window_len"][0])
        self.steps_per_day = int(meta["steps_per_day"][0])
        self.input_len = 2 * self.steps_per_day
        self.target_len = 1 * self.steps_per_day

        stats = np.load(self.stats_path)
        mean = stats["mean"]
        std = stats["std"]
        self.mean = torch.from_numpy(mean).view(1, self.C, 1, 1)
        self.std = torch.from_numpy(std).view(1, self.C, 1, 1)

    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        x = torch.from_numpy(np.array(self.Z[idx], copy=True)).to(self.dtype)
        if self.normalize:
            x = (x - self.mean) / self.std
        return x[:self.input_len], x[self.input_len:self.input_len + self.target_len]  # (Tin,Tout)

# ============================================================
# =============== 3) 公共工具（Utils） =======================
# ============================================================
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def make_run_dir(cfg: TrainConfig) -> Path:
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_name = f"enc_{cfg.enc_cell_type}_dec_{cfg.dec_cell_type}_{ts}"
    run_dir = Path(cfg.out_root) / run_name
    ensure_dir(run_dir / "checkpoints")
    ensure_dir(run_dir / "figs")
    ensure_dir(run_dir / "vis")
    # 保存配置
    with open(run_dir / "config.json", "w", encoding="utf-8") as f:
        json.dump(asdict(cfg), f, ensure_ascii=False, indent=2)
    return run_dir

def per_channel_mse(pred, target):
    # pred/target: (B, T, C, H, W)
    return ((pred - target) ** 2).mean(dim=(0, 1, 3, 4))

def denorm(x_chw: torch.Tensor, mean_c11: torch.Tensor, std_c11: torch.Tensor):
    """
    x_chw: (C,H,W) 在任意设备
    mean_c11/std_c11: (1,C,1,1) 或 (C,1,1,1)，会自动搬到 x_chw.device
    """
    # 确保 mean/std 在同一设备
    mean_c11 = mean_c11.to(x_chw.device)
    std_c11  = std_c11.to(x_chw.device)
    return x_chw * std_c11.squeeze(0) + mean_c11.squeeze(0)


def plot_loss_curves(history, out_dir: Path):
    ensure_dir(out_dir)
    epochs = [h["epoch"] for h in history]
    train_loss = [h["train_loss"] for h in history]
    val_loss = [h["val_loss"] for h in history]

    # 总体
    plt.figure(figsize=(6,4))
    plt.plot(epochs, train_loss, label="train MSE")
    plt.plot(epochs, val_loss, label="val MSE")
    plt.xlabel("Epoch")
    plt.ylabel("MSE")
    plt.title("Overall Loss")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / "overall_loss.png", dpi=180)
    plt.close()

    # 分通道
    if history and history[0]["train_c_loss"]:
        C = len(history[0]["train_c_loss"])
        plt.figure(figsize=(7,5))
        for c in range(C):
            # 用变量名当 legend，如果 VARIABLES 不够长就退回 c 索引
            if c < len(VARIABLES):
                vname = VARIABLES[c]
            else:
                vname = f"var_{c}"

            plt.plot(
                epochs,
                [h["train_c_loss"][c] for h in history],
                "-",
                label=f"train {vname}",
            )
            plt.plot(
                epochs,
                [h["val_c_loss"][c] for h in history],
                "--",
                label=f"val {vname}",
            )
        plt.xlabel("Epoch")
        plt.ylabel("MSE (per channel)")
        plt.title("Per-Channel Loss")
        plt.legend(ncol=2)
        plt.tight_layout()
        plt.savefig(out_dir / "per_channel_loss.png", dpi=180)
        plt.close()


def _choose_t_and_c(Tout: int, C: int, use_last_t: bool, c_fixed: Optional[int]) -> Tuple[int, int]:
    t_idx = (Tout - 1) if use_last_t else int(torch.randint(0, Tout, (1,)).item())
    if c_fixed is None:
        c_idx = int(torch.randint(0, C, (1,)).item())
    else:
        c_idx = int(max(0, min(C - 1, c_fixed)))
    return t_idx, c_idx

def _save_heatmaps(gt_chw: np.ndarray,
                   pd_chw: np.ndarray,
                   save_prefix: Path,
                   draw_all_channels: bool,
                   c_idx: int,
                   var_names: Optional[List[str]] = None):
    """
    gt_chw/pd_chw: (C,H,W) in physical value (already denormalized)

    一张图，两行：
      - 第一行：GT
      - 第二行：Pred
    每一列对应一个通道（变量）
    """
    ensure_dir(save_prefix.parent)

    if draw_all_channels:
        channels = list(range(gt_chw.shape[0]))
    else:
        channels = [c_idx]

    n_ch = len(channels)

    fig, axes = plt.subplots(2, n_ch, figsize=(4 * n_ch, 6), squeeze=False)

    for j, c in enumerate(channels):
        if var_names is not None and c < len(var_names):
            name = var_names[c]
        else:
            name = f"c={c}"

        # 上：GT
        ax_gt = axes[0, j]
        im_gt = ax_gt.imshow(gt_chw[c], origin="lower")
        ax_gt.set_title(f"GT | {name}", fontsize=9)
        ax_gt.axis("off")
        fig.colorbar(im_gt, ax=ax_gt, fraction=0.046, pad=0.04)

        # 下：Pred
        ax_pd = axes[1, j]
        im_pd = ax_pd.imshow(pd_chw[c], origin="lower")
        ax_pd.set_title(f"Pred | {name}", fontsize=9)
        ax_pd.axis("off")
        fig.colorbar(im_pd, ax=ax_pd, fraction=0.046, pad=0.04)

    plt.tight_layout()
    fig.savefig(save_prefix.with_suffix(".png"), dpi=160)
    plt.close(fig)

@torch.no_grad()
def visualize_once(model: nn.Module,
                   x_in: torch.Tensor, y_out: torch.Tensor,
                   mean: torch.Tensor, std: torch.Tensor,
                   save_prefix: Path,
                   use_last_t: bool,
                   viz_channel: Optional[int],
                   draw_all_channels: bool):
    """
    对 batch 里第一条样本，TF=0 自由解码，挑选一个时间步画图。
    画法：一张图，两行（GT / Pred），每列一个通道。
    """
    model_was_training = model.training
    model.eval()

    x1 = x_in[:1]                           # (1, Tin, C, H, W)
    Tout = y_out.shape[1]
    pred1 = model(x1, y_len=Tout, teacher_forcing_ratio=0.0, targets=None)  # (1, Tout, C, H, W)

    t_idx, c_idx = _choose_t_and_c(Tout, x1.shape[2], use_last_t, viz_channel)
    gt_chw = y_out[0, t_idx]                # (C,H,W)
    pd_chw = pred1[0, t_idx]                # (C,H,W)

    gt_phys = denorm(gt_chw.cpu(), mean, std).numpy()
    pd_phys = denorm(pd_chw.cpu(), mean, std).numpy()

    _save_heatmaps(gt_phys, pd_phys, save_prefix,
                   draw_all_channels, c_idx,
                   var_names=VARIABLES)

    if model_was_training:
        model.train()

# ============================================================
# =============== 4) 模型（Model） ===========================
# ============================================================
def layer_norm_2d(num_channels: int) -> nn.Module:
    return nn.GroupNorm(1, num_channels)

class ConvRNNCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, layernorm: bool = False):
        super().__init__()
        p = kernel_size // 2
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.ln = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        return torch.tanh(self.ln(self.conv(torch.cat([x, h_prev], dim=1))))

    def init_hidden(self, B, H, W, device=None, dtype=None):
        return torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)

class ConvGRUCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, layernorm: bool = False):
        super().__init__()
        p = kernel_size // 2
        self.hidden_dim = hidden_dim
        self.z = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.r = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.h = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.ln_z = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_r = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_h = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        xh = torch.cat([x, h_prev], dim=1)
        z = torch.sigmoid(self.ln_z(self.z(xh)))
        r = torch.sigmoid(self.ln_r(self.r(xh)))
        xrh = torch.cat([x, r * h_prev], dim=1)
        h_tilde = torch.tanh(self.ln_h(self.h(xrh)))
        return (1 - z) * h_prev + z * h_tilde

    def init_hidden(self, B, H, W, device=None, dtype=None):
        return torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, layernorm: bool = False, peephole: bool = False):
        super().__init__()
        p = kernel_size // 2
        self.hidden_dim = hidden_dim
        self.peephole = peephole
        self.conv = nn.Conv2d(input_dim + hidden_dim, 4 * hidden_dim, kernel_size, padding=p, bias=bias)
        self.ln_i = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_f = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_o = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_g = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        if peephole:
            self.Wci = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))
            self.Wcf = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))
            self.Wco = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor, c_prev: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        gates = self.conv(torch.cat([x, h_prev], dim=1))
        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)
        i = self.ln_i(i); f = self.ln_f(f); o = self.ln_o(o); g = self.ln_g(g)
        if self.peephole:
            i = torch.sigmoid(i + self.Wci * c_prev)
            f = torch.sigmoid(f + self.Wcf * c_prev)
        else:
            i = torch.sigmoid(i); f = torch.sigmoid(f)
        g = torch.tanh(g)
        c = f * c_prev + i * g
        o = torch.sigmoid(o + self.Wco * c) if self.peephole else torch.sigmoid(o)
        h = o * torch.tanh(c)
        return h, c

    def init_hidden(self, B, H, W, device=None, dtype=None) -> Tuple[torch.Tensor, torch.Tensor]:
        h = torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)
        c = torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)
        return h, c

# --- 轻量 U-Net 头 ---
class ResidualDoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch, norm='gn'):
        super().__init__()
        Norm = (lambda c: nn.GroupNorm(1, c)) if norm=='gn' else (lambda c: nn.BatchNorm2d(c))
        self.conv1 = nn.Conv2d(in_ch,  out_ch, 3, padding=1, bias=False)
        self.n1    = Norm(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False)
        self.n2    = Norm(out_ch)
        self.act   = nn.ReLU(inplace=True)
        self.proj  = nn.Identity() if in_ch == out_ch else nn.Conv2d(in_ch, out_ch, 1, bias=False)

    def forward(self, x):
        identity = self.proj(x)
        out = self.act(self.n1(self.conv1(x)))
        out = self.n2(self.conv2(out))
        out = self.act(out + identity)
        return out

def _pad_to_multiple(x: torch.Tensor, multiple: int) -> Tuple[torch.Tensor, Tuple[int,int]]:
    """
    把特征图 pad 到 H、W 都是 multiple 的倍数。
    返回 (x_padded, (pad_h, pad_w))，方便最后裁剪回原尺寸。
    """
    _, _, H, W = x.shape
    pad_h = (-H) % multiple
    pad_w = (-W) % multiple
    if pad_h or pad_w:
        # F.pad 的顺序是 (left, right, top, bottom)
        x = F.pad(x, (0, pad_w, 0, pad_h), mode="constant", value=0)
    return x, (pad_h, pad_w)

class UNetHead(nn.Module):
    """
    尺寸安全版 U-Net 头（levels=2 下/上采样两次）。
    - 进入时把输入 pad 到 4 的倍数，退出时裁回原始 H、W
    - 其他结构与原轻量 U-Net 相同
    """
    def __init__(self, in_ch: int, out_ch: int, base: int = 64, levels: int = 2):
        super().__init__()
        assert levels == 2, "当前实现假设两层下采样（可按需扩展）"
        b = min(base, max(32, in_ch))

        # Encoder
        self.enc1 = ResidualDoubleConv(in_ch, b)
        self.pool1 = nn.MaxPool2d(2)              # H/2, W/2
        self.enc2 = ResidualDoubleConv(b, b * 2)
        self.pool2 = nn.MaxPool2d(2)              # H/4, W/4

        # Bottleneck
        self.mid = ResidualDoubleConv(b * 2, b * 4)

        # Decoder
        self.up2  = nn.ConvTranspose2d(b * 4, b * 2, 2, stride=2)  # ×2
        self.dec2 = ResidualDoubleConv(b * 4, b * 2)
        self.up1  = nn.ConvTranspose2d(b * 2, b, 2, stride=2)      # ×2
        self.dec1 = ResidualDoubleConv(b * 2, b)

        # Output
        self.out = nn.Conv2d(b, out_ch, 1)

    @staticmethod
    def _match_and_cat(skip: torch.Tensor, up: torch.Tensor) -> torch.Tensor:
        """
        由于上/下采样后可能有 1 个像素的差异，这里把 skip 做中心裁剪以匹配 up 的空间尺寸。
        """
        _, _, Hs, Ws = skip.shape
        _, _, Hu, Wu = up.shape
        dh = Hs - Hu
        dw = Ws - Wu
        if dh or dw:
            top = dh // 2
            left = dw // 2
            skip = skip[:, :, top:top+Hu, left:left+Wu]
        return torch.cat([up, skip], dim=1)

    def forward(self, x):
        B, C, H, W = x.shape
        # 1) pad 到 4 的倍数，保证两次池化/反卷积后尺寸严格对齐
        x, (pad_h, pad_w) = _pad_to_multiple(x, multiple=4)

        # Encoder
        e1 = self.enc1(x)              # H,   W
        e2 = self.enc2(self.pool1(e1)) # H/2, W/2

        # Mid
        mid = self.mid(self.pool2(e2)) # H/4, W/4

        # Decoder
        u2  = self.up2(mid)            # H/2, W/2
        d2  = self.dec2(self._match_and_cat(e2, u2))

        u1  = self.up1(d2)             # H,   W
        d1  = self.dec1(self._match_and_cat(e1, u1))

        out = self.out(d1)

        # 2) 裁回原始 H、W（把右侧/下侧多出来的 pad 去掉）
        if pad_h or pad_w:
            out = out[:, :, :H, :W]
        return out

CellType = Union[ConvRNNCell, ConvGRUCell, ConvLSTMCell]

def make_cell(cell_type: str, in_ch: int, hid_ch: int, kernel_size: int,
              layernorm: bool, peephole: bool) -> CellType:
    t = cell_type.lower()
    if t == "convrnn":
        return ConvRNNCell(in_ch, hid_ch, kernel_size, layernorm=layernorm)
    elif t == "convgru":
        return ConvGRUCell(in_ch, hid_ch, kernel_size, layernorm=layernorm)
    elif t == "convlstm":
        return ConvLSTMCell(in_ch, hid_ch, kernel_size, layernorm=layernorm, peephole=peephole)
    else:
        raise ValueError(f"Unknown cell_type: {cell_type}")

class RecurrentStack(nn.Module):
    def __init__(self, cell_type: str, input_dim: int, hidden_dims: List[int],
                 kernel_size: int = 3, layernorm: bool = False, peephole: bool = False):
        super().__init__()
        self.cell_type = cell_type.lower()
        self.hidden_dims = hidden_dims
        self.layers = nn.ModuleList()
        ch_in = input_dim
        for hid in hidden_dims:
            self.layers.append(make_cell(self.cell_type, ch_in, hid, kernel_size, layernorm, peephole))
            ch_in = hid

    def init_state(self, B, H, W, device=None, dtype=None):
        states = []
        for cell in self.layers:
            states.append(cell.init_hidden(B, H, W, device=device, dtype=dtype))
        return states

    def forward(self, x, states):
        new_states = []
        h_in = x
        for layer, state in zip(self.layers, states):
            if isinstance(layer, ConvLSTMCell):
                h_prev, c_prev = state
                h, c = layer(h_in, h_prev, c_prev)
                new_states.append((h, c))
                h_in = h
            else:
                h_prev = state
                h = layer(h_in, h_prev)
                new_states.append(h)
                h_in = h
        return h_in, new_states

class ConvSeq2Seq(nn.Module):
    def __init__(self,
                 input_dim: int,
                 enc_cell_type: str = "convlstm",
                 dec_cell_type: str = "convlstm",
                 hidden_dims_enc: List[int] = [32],
                 hidden_dims_dec: List[int] = [32],
                 kernel_size: int = 3,
                 layernorm: bool = False,
                 peephole: bool = False,
                 unet_head: bool = False):
        super().__init__()
        self.input_dim = input_dim
        self.enc = RecurrentStack(enc_cell_type, input_dim, hidden_dims_enc,
                                  kernel_size=kernel_size, layernorm=layernorm, peephole=peephole)
        self.dec = RecurrentStack(dec_cell_type, input_dim, hidden_dims_dec,
                                  kernel_size=kernel_size, layernorm=layernorm, peephole=peephole)
        dec_top = hidden_dims_dec[-1]
        self.head = UNetHead(dec_top, input_dim) if unet_head else nn.Conv2d(dec_top, input_dim, 1)

    def forward(self, x_in: torch.Tensor, y_len: int, teacher_forcing_ratio: float = 0.5,
                targets: Optional[torch.Tensor] = None):
        B, Tin, C, H, W = x_in.shape
        device, dtype = x_in.device, x_in.dtype

        # Encode
        enc_states = self.enc.init_state(B, H, W, device=device, dtype=dtype)
        h = None
        for t in range(Tin):
            h, enc_states = self.enc(x_in[:, t], enc_states)

        # Bridge: enc -> dec
        dec_states = []
        for enc_state, dec_cell in zip(enc_states, self.dec.layers):
            if isinstance(dec_cell, ConvLSTMCell):
                if isinstance(enc_state, tuple):
                    h0, c0 = enc_state
                else:
                    h0 = enc_state
                    c0 = torch.zeros_like(h0)
                dec_states.append((h0, c0))
            else:
                h0 = enc_state[0] if isinstance(enc_state, tuple) else enc_state
                dec_states.append(h0)

        # Decode
        y = []
        dec_in = x_in[:, -1]
        for t in range(y_len):
            top_h, dec_states = self.dec(dec_in, dec_states)
            out = self.head(top_h)
            y.append(out.unsqueeze(1))
            use_tf = (targets is not None) and (torch.rand((), device=device).item() < teacher_forcing_ratio) and (t < targets.shape[1])
            dec_in = targets[:, t] if use_tf else out.detach()
        return torch.cat(y, dim=1)

# ============================================================
# =============== 5) 训练（Train/Eval/Viz） ==================
# ============================================================
def get_datasets(cfg: TrainConfig):
    full_ds = ERA5ZInOutDataset(
        data_dir=cfg.data_dir,
        z_name=cfg.z_name,
        meta_name=cfg.meta_name,
        stats_name=cfg.stats_name,
        normalize=True,
    )
    n_total = len(full_ds)
    n_train = int(0.8 * n_total)
    n_val = n_total - n_train
    train_ds, val_ds = random_split(full_ds, [n_train, n_val])
    print(f"数据集总数: {n_total} | 训练集: {n_train} | 验证集: {n_val}")
    return train_ds, val_ds

def _tf_ratio(cfg: TrainConfig, epoch: int) -> float:
    if cfg.enable_tf_ratio == False:
        return 1
    else:
        if cfg.max_epochs <= 1:
            return cfg.teacher_forcing_end
        t = epoch / (cfg.max_epochs - 1)
        return (1 - t) * cfg.teacher_forcing_start + t * cfg.teacher_forcing_end

def _build_model(cfg: TrainConfig, C: int, device: torch.device) -> nn.Module:
    hid_enc = [cfg.hidden_dim] * cfg.n_enc_layers
    hid_dec = [cfg.hidden_dim] * cfg.n_dec_layers
    model = ConvSeq2Seq(
        input_dim=C,
        enc_cell_type=cfg.enc_cell_type,
        dec_cell_type=cfg.dec_cell_type,
        hidden_dims_enc=hid_enc,
        hidden_dims_dec=hid_dec,
        kernel_size=cfg.kernel_size,
        layernorm=cfg.layernorm,
        peephole=cfg.peephole,
        unet_head=cfg.unet_head,
    ).to(device)
    return model

def _save_history_artifacts(run_dir: Path, cfg: TrainConfig, history):
    # JSON
    with open(run_dir / "history.json", "w", encoding="utf-8") as f:
        json.dump({
            "config": asdict(cfg),
            "history": history,
        }, f, ensure_ascii=False, indent=2)

    # CSV
    if history:
        C = len(history[0]["train_c_loss"])
        fieldnames = [
            "epoch",
            "train_loss", "val_loss",
            "train_rmse", "val_rmse",
            "train_mae", "val_mae",
            "train_bias", "val_bias",
        ] + [f"train_c{c}" for c in range(C)] + [f"val_c{c}" for c in range(C)]

        with open(run_dir / "history.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for h in history:
                row = {
                    "epoch": h["epoch"],
                    "train_loss": h["train_loss"],
                    "val_loss": h["val_loss"],
                    "train_rmse": h.get("train_rmse", float("nan")),
                    "val_rmse": h.get("val_rmse", float("nan")),
                    "train_mae": h.get("train_mae", float("nan")),
                    "val_mae": h.get("val_mae", float("nan")),
                    "train_bias": h.get("train_bias", float("nan")),
                    "val_bias": h.get("val_bias", float("nan")),
                }
                for c in range(C):
                    row[f"train_c{c}"] = h["train_c_loss"][c]
                    row[f"val_c{c}"] = h["val_c_loss"][c]
                writer.writerow(row)

        # NPZ
        np.savez_compressed(
            run_dir / "history.npz",
            epoch=np.array([h["epoch"] for h in history], dtype=np.int32),
            train_loss=np.array([h["train_loss"] for h in history], dtype=np.float64),
            val_loss=np.array([h["val_loss"] for h in history], dtype=np.float64),
            train_rmse=np.array([h.get("train_rmse", np.nan) for h in history], dtype=np.float64),
            val_rmse=np.array([h.get("val_rmse", np.nan) for h in history], dtype=np.float64),
            train_mae=np.array([h.get("train_mae", np.nan) for h in history], dtype=np.float64),
            val_mae=np.array([h.get("val_mae", np.nan) for h in history], dtype=np.float64),
            train_bias=np.array([h.get("train_bias", np.nan) for h in history], dtype=np.float64),
            val_bias=np.array([h.get("val_bias", np.nan) for h in history], dtype=np.float64),
            train_c_loss=np.stack([np.array(h["train_c_loss"], dtype=np.float64) for h in history], axis=0),
            val_c_loss=np.stack([np.array(h["val_c_loss"], dtype=np.float64) for h in history], axis=0),
        )


def train(cfg: TrainConfig,
          train_loader: DataLoader,
          val_loader: DataLoader,
          mean_c11: torch.Tensor,
          std_c11: torch.Tensor):
    """
    版本：不在内部创建 Dataset/DataLoader，
          一切数据都由外部准备好，然后传进来。

    参数：
      - cfg: 训练配置
      - train_loader, val_loader: 外部构建好的 DataLoader
        * 要求 batch 输出是 (x_in, y_out)，
          形状分别为 (B, Tin, C, H, W) / (B, Tout, C, H, W)
      - mean_c11, std_c11: 形状 (1, C, 1, 1)，用于反归一化可视化
    """
    # ---- 设备与输出目录 ----
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    run_dir = make_run_dir(cfg)
    ckpt_dir = run_dir / "checkpoints"
    figs_dir = run_dir / "figs"
    vis_dir = run_dir / "vis"

    pin = torch.cuda.is_available()

    # ---- 通道数 C ----
    # 从 mean/std 推出即可：mean_c11: (1,C,1,1)
    C = mean_c11.shape[1]

    # ---- 模型/优化器 ----
    model = _build_model(cfg, C, device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr)
    criterion = nn.MSELoss()

    # ---- 训练循环 ----
    history = []
    best_val = float("inf")

    for epoch in range(cfg.max_epochs):
        model.train()
        tf = _tf_ratio(cfg, epoch)

        tr_loss_sum = 0.0
        tr_c_loss_sum = None
        tr_mae_sum = 0.0
        tr_bias_sum = 0.0

        for x_in, y_out in train_loader:
            x_in = x_in.to(device, non_blocking=True)
            y_out = y_out.to(device, non_blocking=True)

            pred = model(
                x_in,
                y_len=y_out.shape[1],
                teacher_forcing_ratio=tf,
                targets=y_out,
            )
            loss = criterion(pred, y_out)
            diff = pred - y_out

            with torch.no_grad():
                # per-channel MSE（标准化空间）
                c_loss_batch = per_channel_mse(pred, y_out)
                if tr_c_loss_sum is None:
                    tr_c_loss_sum = c_loss_batch * x_in.size(0)
                else:
                    tr_c_loss_sum += c_loss_batch * x_in.size(0)

                # overall MAE & Bias（按 sample 数加权）
                mae_batch = diff.abs().mean().item()
                bias_batch = diff.mean().item()
                tr_mae_sum += mae_batch * x_in.size(0)
                tr_bias_sum += bias_batch * x_in.size(0)

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
            optimizer.step()

            tr_loss_sum += loss.item() * x_in.size(0)

        # ---- 聚合 train 指标 ----
        n_train = len(train_loader.dataset)
        tr_loss = tr_loss_sum / n_train
        tr_c_loss = (tr_c_loss_sum / n_train).detach().cpu().tolist() if tr_c_loss_sum is not None else []
        tr_mae = tr_mae_sum / n_train
        tr_bias = tr_bias_sum / n_train
        tr_rmse = tr_loss ** 0.5

        # ---- 验证 ----
        model.eval()
        val_loss_sum = 0.0
        val_c_loss_sum = None
        val_mae_sum = 0.0
        val_bias_sum = 0.0

        with torch.no_grad():
            for x_in, y_out in val_loader:
                x_in = x_in.to(device, non_blocking=True)
                y_out = y_out.to(device, non_blocking=True)
                pred = model(
                    x_in,
                    y_len=y_out.shape[1],
                    teacher_forcing_ratio=0.0,
                    targets=None,
                )
                loss = criterion(pred, y_out)
                val_loss_sum += loss.item() * x_in.size(0)

                diff = pred - y_out
                c_loss_batch = per_channel_mse(pred, y_out)
                if val_c_loss_sum is None:
                    val_c_loss_sum = c_loss_batch * x_in.size(0)
                else:
                    val_c_loss_sum += c_loss_batch * x_in.size(0)

                mae_batch = diff.abs().mean().item()
                bias_batch = diff.mean().item()
                val_mae_sum += mae_batch * x_in.size(0)
                val_bias_sum += bias_batch * x_in.size(0)

        n_val = len(val_loader.dataset)
        val_loss = val_loss_sum / n_val
        val_c_loss = (val_c_loss_sum / n_val).detach().cpu().tolist() if val_c_loss_sum is not None else []
        val_mae = val_mae_sum / n_val
        val_bias = val_bias_sum / n_val
        val_rmse = val_loss ** 0.5

        print(
            f"[Epoch {epoch+1}/{cfg.max_epochs}] "
            f"TF={tf:.2f} | "
            f"train MSE={tr_loss:.6f} RMSE={tr_rmse:.6f} MAE={tr_mae:.6f} | "
            f"val MSE={val_loss:.6f} RMSE={val_rmse:.6f} MAE={val_mae:.6f}"
        )

        history.append({
            "epoch": epoch + 1,
            "train_loss": float(tr_loss),
            "val_loss": float(val_loss),
            "train_rmse": float(tr_rmse),
            "val_rmse": float(val_rmse),
            "train_mae": float(tr_mae),
            "val_mae": float(val_mae),
            "train_bias": float(tr_bias),
            "val_bias": float(val_bias),
            "train_c_loss": [float(x) for x in tr_c_loss],
            "val_c_loss": [float(x) for x in val_c_loss],
        })

        # ---- 可视化（train & eval，每隔 n 个 epoch）----
        if cfg.viz_every_n_epochs and ((epoch + 1) % cfg.viz_every_n_epochs == 0):
            # 从原始 dataset 中随机取一条可视化
            train_batch = next(iter(DataLoader(train_loader.dataset, batch_size=1, shuffle=True, pin_memory=pin)))
            eval_batch  = next(iter(DataLoader(val_loader.dataset,   batch_size=1, shuffle=True, pin_memory=pin)))
            tx, ty = train_batch[0].to(device), train_batch[1].to(device)
            ex, ey = eval_batch[0].to(device),  eval_batch[1].to(device)
            epoch_dir = vis_dir / f"epoch_{epoch+1:04d}"

            visualize_once(
                model, tx, ty, mean_c11, std_c11,
                save_prefix=epoch_dir / "train_sample",
                use_last_t=cfg.viz_use_last_t,
                viz_channel=cfg.viz_channel,
                draw_all_channels=cfg.save_all_channels_png,
            )
            visualize_once(
                model, ex, ey, mean_c11, std_c11,
                save_prefix=epoch_dir / "eval_sample",
                use_last_t=cfg.viz_use_last_t,
                viz_channel=cfg.viz_channel,
                draw_all_channels=cfg.save_all_channels_png,
            )

        # ---- 保存 checkpoint ----
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), ckpt_dir / "best.pt")
            print(f"  >> Saved best model (val MSE={best_val:.6f})")
        torch.save(model.state_dict(), ckpt_dir / "last.pt")

    # ---- 训练结束：落盘历史 + 曲线 ----
    _save_history_artifacts(run_dir, cfg, history)
    plot_loss_curves(history, figs_dir)

    return model, history


# def train(cfg: TrainConfig):
#     # ---- 设备与输出目录 ----
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     run_dir = make_run_dir(cfg)
#     ckpt_dir = run_dir / "checkpoints"
#     figs_dir = run_dir / "figs"
#     vis_dir = run_dir / "vis"

#     # ---- 数据 ----
#     train_ds, val_ds = get_datasets(cfg)
#     mean_c11 = train_ds.dataset.mean
#     std_c11  = train_ds.dataset.std
#     pin = torch.cuda.is_available()
#     train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True,
#                               num_workers=cfg.num_workers, pin_memory=pin)
#     val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False,
#                             num_workers=cfg.num_workers, pin_memory=pin)

#     # ---- 模型/优化器 ----
#     C = train_ds.dataset.C
#     model = _build_model(cfg, C, device)
#     optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr)
#     criterion = nn.MSELoss()

#     # ---- 训练循环 ----
#     history = []
#     best_val = float("inf")

#     for epoch in range(cfg.max_epochs):
#         model.train()
#         tf = _tf_ratio(cfg, epoch)

#         tr_loss_sum = 0.0
#         tr_c_loss_sum = None
#         tr_mae_sum = 0.0
#         tr_bias_sum = 0.0

#         for x_in, y_out in train_loader:
#             x_in = x_in.to(device, non_blocking=True)
#             y_out = y_out.to(device, non_blocking=True)

#             pred = model(x_in, y_len=y_out.shape[1], teacher_forcing_ratio=tf, targets=y_out)
#             loss = criterion(pred, y_out)

#             diff = pred - y_out

#             with torch.no_grad():
#                 # per-channel MSE
#                 c_loss_batch = per_channel_mse(pred, y_out)
#                 if tr_c_loss_sum is None:
#                     tr_c_loss_sum = c_loss_batch * x_in.size(0)
#                 else:
#                     tr_c_loss_sum += c_loss_batch * x_in.size(0)

#                 # overall MAE & Bias（按 sample 数加权）
#                 mae_batch = diff.abs().mean().item()
#                 bias_batch = diff.mean().item()
#                 tr_mae_sum += mae_batch * x_in.size(0)
#                 tr_bias_sum += bias_batch * x_in.size(0)

#             optimizer.zero_grad(set_to_none=True)
#             loss.backward()
#             nn.utils.clip_grad_norm_(model.parameters(), cfg.grad_clip)
#             optimizer.step()

#             tr_loss_sum += loss.item() * x_in.size(0)

#         tr_loss = tr_loss_sum / len(train_loader.dataset)
#         tr_c_loss = (tr_c_loss_sum / len(train_loader.dataset)).detach().cpu().tolist() if tr_c_loss_sum is not None else []
#         tr_mae = tr_mae_sum / len(train_loader.dataset)
#         tr_bias = tr_bias_sum / len(train_loader.dataset)
#         tr_rmse = tr_loss ** 0.5

#         # ---- 验证 ----
#         model.eval()
#         val_loss_sum = 0.0
#         val_c_loss_sum = None
#         val_mae_sum = 0.0
#         val_bias_sum = 0.0

#         with torch.no_grad():
#             for x_in, y_out in val_loader:
#                 x_in = x_in.to(device, non_blocking=True)
#                 y_out = y_out.to(device, non_blocking=True)
#                 pred = model(x_in, y_len=y_out.shape[1], teacher_forcing_ratio=0.0, targets=None)
#                 loss = criterion(pred, y_out)
#                 val_loss_sum += loss.item() * x_in.size(0)

#                 diff = pred - y_out

#                 c_loss_batch = per_channel_mse(pred, y_out)
#                 if val_c_loss_sum is None:
#                     val_c_loss_sum = c_loss_batch * x_in.size(0)
#                 else:
#                     val_c_loss_sum += c_loss_batch * x_in.size(0)

#                 mae_batch = diff.abs().mean().item()
#                 bias_batch = diff.mean().item()
#                 val_mae_sum += mae_batch * x_in.size(0)
#                 val_bias_sum += bias_batch * x_in.size(0)

#         val_loss = val_loss_sum / len(val_loader.dataset)
#         val_c_loss = (val_c_loss_sum / len(val_loader.dataset)).detach().cpu().tolist() if val_c_loss_sum is not None else []
#         val_mae = val_mae_sum / len(val_loader.dataset)
#         val_bias = val_bias_sum / len(val_loader.dataset)
#         val_rmse = val_loss ** 0.5

#         print(
#             f"[Epoch {epoch+1}/{cfg.max_epochs}] "
#             f"TF={tf:.2f} | "
#             f"train MSE={tr_loss:.6f} RMSE={tr_rmse:.6f} MAE={tr_mae:.6f} | "
#             f"val MSE={val_loss:.6f} RMSE={val_rmse:.6f} MAE={val_mae:.6f}"
#         )

#         history.append({
#             "epoch": epoch + 1,
#             "train_loss": float(tr_loss),
#             "val_loss": float(val_loss),
#             "train_rmse": float(tr_rmse),
#             "val_rmse": float(val_rmse),
#             "train_mae": float(tr_mae),
#             "val_mae": float(val_mae),
#             "train_bias": float(tr_bias),
#             "val_bias": float(val_bias),
#             "train_c_loss": [float(x) for x in tr_c_loss],
#             "val_c_loss": [float(x) for x in val_c_loss],
#         })

#         # ---- 可视化（train & eval，每隔 n 个 epoch）----
#         if cfg.viz_every_n_epochs and ((epoch + 1) % cfg.viz_every_n_epochs == 0):
#             # 取一个小 batch
#             train_batch = next(iter(DataLoader(train_ds, batch_size=1, shuffle=True)))
#             eval_batch  = next(iter(DataLoader(val_ds,   batch_size=1, shuffle=True)))
#             tx, ty = train_batch[0].to(device), train_batch[1].to(device)
#             ex, ey = eval_batch[0].to(device),  eval_batch[1].to(device)
#             epoch_dir = vis_dir / f"epoch_{epoch+1:04d}"
#             # train 可视化
#             visualize_once(
#                 model, tx, ty, mean_c11, std_c11,
#                 save_prefix=epoch_dir / "train_sample",
#                 use_last_t=cfg.viz_use_last_t,
#                 viz_channel=cfg.viz_channel,
#                 draw_all_channels=cfg.save_all_channels_png
#             )
#             # eval 可视化
#             visualize_once(
#                 model, ex, ey, mean_c11, std_c11,
#                 save_prefix=epoch_dir / "eval_sample",
#                 use_last_t=cfg.viz_use_last_t,
#                 viz_channel=cfg.viz_channel,
#                 draw_all_channels=cfg.save_all_channels_png
#             )

#         # ---- 保存 checkpoint ----
#         if val_loss < best_val:
#             best_val = val_loss
#             torch.save(model.state_dict(), ckpt_dir / "best.pt")
#             print(f"  >> Saved best model (val MSE={best_val:.6f})")
#         torch.save(model.state_dict(), ckpt_dir / "last.pt")

#     # ---- 训练结束：落盘历史 + 曲线 ----
#     _save_history_artifacts(run_dir, cfg, history)
#     plot_loss_curves(history, figs_dir)

#     return model, history

# ============================================================
# =============== 6) 入口（Main） ============================
# ============================================================
# if __name__ == "__main__":
#     cfg = TrainConfig(
#         data_dir="./",
#         max_epochs=100,
#         batch_size=2,
#         enc_cell_type="convgru",
#         dec_cell_type="convgru",
#         n_enc_layers=1,
#         n_dec_layers=1,
#         enable_tf_ratio=False,
#         layernorm=True,
#         peephole=False,
#         unet_head=True,
#         out_root="./runs_w_unet",
#         viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
#         viz_use_last_t=True,       # 可切换 False 以随机时间步
#         viz_channel=None,          # None=随机通道；也可设 0/1/2...
#         save_all_channels_png=True
#     )
#     model, history = train(cfg)


#### metrics

##### weighting

In [ ]:
def lat_centers(num_lat: int, device=None) -> torch.Tensor:
    j = torch.arange(num_lat, device=device, dtype=torch.float32)
    dphi = 180.0 / float(num_lat - 1)
    phi_center = 90.0 - j * dphi
    return phi_center  # degrees


def latitude_area_weights(num_lat: int, device=None) -> torch.Tensor:
    """
    Compute w(i) ∝ sin(theta_u) - sin(theta_l), normalized to sum to 1.
    """
    phi_center = lat_centers(num_lat, device=device)
    dphi = 180.0 / float(num_lat - 1)

    phi_upper_deg = torch.clamp(phi_center + 0.5 * dphi, max=90.0)
    phi_lower_deg = torch.clamp(phi_center - 0.5 * dphi, min=-90.0)

    phi_upper = torch.deg2rad(phi_upper_deg)
    phi_lower = torch.deg2rad(phi_lower_deg)

    w_raw = torch.sin(phi_upper) - torch.sin(phi_lower)  # [H]
    w = w_raw / w_raw.sum()
    return w  # [H]

##### RMSE

In [ ]:
def weighted_rmse_channels(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Compute latitude-area-weighted RMSE per channel.

    Args:
        pred, target: tensors of shape [B, C, H, W]
                      (batch, channels, latitude, longitude)

    Returns:
        rmse: tensor of shape [B, C]
              (one RMSE per channel for each batch sample)
    """
    assert pred.shape == target.shape, "pred and target must have same shape"
    assert pred.dim() == 4, "expected shape [B, C, H, W]"

    B, C, H, W = pred.shape
    device = pred.device

    err2 = (pred - target) ** 2  # [B, C, H, W]

    # latitude weights w_i (sum to 1)
    w_lat = latitude_area_weights(H, device=device)     # [H]
    w_lat = w_lat.view(1, 1, H, 1)                      # [1,1,H,1]

    # mean over longitude, then weighted sum over latitude
    # mean over W: [B, C, H]
    err2_mean_lon = err2.mean(dim=-1)                   # [B, C, H]

    # weighted sum over H (since sum_i w_i = 1)
    mse = (w_lat.squeeze(-1) * err2_mean_lon).sum(dim=-1)  # [B, C]

    rmse = torch.sqrt(mse)
    return rmse

##### ACC

In [ ]:
def weighted_acc_channels(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Latitude-area-weighted ACC (anomaly correlation / pattern correlation)
    per channel.

    pred, target: [B, C, H, W]  (batch, channels, lat, lon)

    Returns:
        acc: [B, C]
    """
    assert pred.shape == target.shape
    assert pred.dim() == 4

    B, C, H, W = pred.shape
    device = pred.device

    # latitude area weights w(j) based on sin(theta_u) - sin(theta_l)
    w_lat = latitude_area_weights(H, device=device)   # [H]
    w = w_lat.view(1, 1, H, 1)                        # [1, 1, H, 1] for broadcasting

    # numerator: weighted dot product
    num = torch.sum(w * pred * target, dim=(-1, -2))  # [B, C]

    # denominator: product of weighted norms
    denom = torch.sqrt(
        torch.sum(w * pred * pred, dim=(-1, -2)) *
        torch.sum(w * target * target, dim=(-1, -2)) + 1e-12
    )  # [B, C]

    acc = num / denom
    return acc

##### MAE

In [ ]:
def weighted_mae_channels(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Latitude-area-weighted MAE per channel.

    Args:
        pred, target: [B, C, H, W]  (batch, channels, lat, lon)

    Returns:
        mae: [B, C]  (MAE for each batch sample and channel)
    """
    assert pred.shape == target.shape, "pred and target must have same shape"
    assert pred.dim() == 4, "expected shape [B, C, H, W]"

    B, C, H, W = pred.shape
    device = pred.device

    err_abs = torch.abs(pred - target)          # [B, C, H, W]

    # latitude weights w_i (sum to 1)
    w_lat = latitude_area_weights(H, device=device)  # [H]
    w_lat = w_lat.view(1, 1, H)                      # [1, 1, H]

    # mean over longitude (uniform grid)
    err_abs_mean_lon = err_abs.mean(dim=-1)         # [B, C, H]

    # weighted sum over latitude
    mae = (w_lat * err_abs_mean_lon).sum(dim=-1)    # [B, C]

    return mae

##### RMSB

In [ ]:
def weighted_rmsb_channels(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Compute globally averaged root mean squared bias (RMSB) per channel.

    Args:
        pred, target: tensors of shape [T, C, H, W]
                      T = time, C = channels/variables,
                      H = latitude, W = longitude.

    Returns:
        rmsb: tensor of shape [C]  (RMSB for each channel)
    """
    assert pred.shape == target.shape, "pred and target must have same shape"
    assert pred.dim() == 4, "expected shape [T, C, H, W]"

    T, C, H, W = pred.shape
    device = pred.device

    # 1) Bias_{i,j} per channel: average over time
    #    Bias[c, i, j] = (1/T) Σ_t (pred - target)
    bias = torch.mean(pred - target, dim=0)      # [C, H, W]

    # 2) Latitude area weights w(i)
    w_lat = latitude_area_weights(H, device=device)  # [H]
    w = w_lat.view(1, H, 1)                          # [1, H, 1] for broadcasting

    # 3) Weighted mean of squared bias over lat & lon
    #    Here we do: Σ_i Σ_j w(i) * Bias^2 / W
    #    (w(i) is normalized to sum to 1, so this is
    #     consistent with area weighting + mean over lon.)
    msb = torch.mean(w * bias**2, dim=(-1, -2))  # [C]

    # 4) Root mean squared bias
    rmsb = torch.sqrt(msb)                       # [C]

    return rmsb


In [ ]:
def weighted_rmsb_channels(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Compute globally averaged root mean squared bias (RMSB) per channel.

    Args:
        pred, target: tensors of shape [T, C, H, W]
                      T = time, C = channels/variables,
                      H = latitude, W = longitude.

    Returns:
        rmsb: tensor of shape [C]  (RMSB for each channel)
    """
    assert pred.shape == target.shape, "pred and target must have same shape"
    assert pred.dim() == 4, "expected shape [T, C, H, W]"

    T, C, H, W = pred.shape
    device = pred.device

    # 1) Bias_{i,j} per channel: average over time
    #    Bias[c, i, j] = (1/T) Σ_t (pred - target)
    bias = torch.mean(pred - target, dim=0)      # [C, H, W]

    # 2) Latitude area weights w(i)
    w_lat = latitude_area_weights(H, device=device)  # [H]
    w = w_lat.view(1, H, 1)                          # [1, H, 1] for broadcasting

    # 3) Weighted mean of squared bias over lat & lon
    #    Here we do: Σ_i Σ_j w(i) * Bias^2 / W
    #    (w(i) is normalized to sum to 1, so this is
    #     consistent with area weighting + mean over lon.)
    msb = torch.mean(w * bias**2, dim=(-1, -2))  # [C]

    # 4) Root mean squared bias
    rmsb = torch.sqrt(msb)                       # [C]

    return rmsb


#### evaluation

In [ ]:
import torch
import numpy as np
from pathlib import Path

# =========================
# 0. mean/std 处理 & 反归一化
# =========================

def get_mean_std_c11(ds_train):
    """
    把 ds_train.mean/std 统一整理成 (1, C, 1, 1) 形状，方便反归一化。
    """
    mean = ds_train.mean
    std  = ds_train.std

    if mean.ndim != 4 or std.ndim != 4:
        raise ValueError(f"Unexpected mean/std shape: mean={mean.shape}, std={std.shape}")

    # 常见两种： (C,1,1,1) 或 (1,C,1,1)
    if mean.shape[0] == 1:          # (1,C,1,1)
        mean_c11 = mean.clone()
        std_c11  = std.clone()
    elif mean.shape[1] == 1:        # (C,1,1,1)
        mean_c11 = mean.permute(1, 0, 2, 3).contiguous()  # -> (1,C,1,1)
        std_c11  = std.permute(1, 0, 2, 3).contiguous()
    else:
        raise ValueError(f"Cannot infer channel dim from mean shape {mean.shape}")

    return mean_c11, std_c11


def denorm_all_channels(norm_tensor, mean_c11, std_c11):
    """
    norm_tensor: (..., C, H, W) 或 (B,T,C,H,W)
    mean_c11/std_c11: (1,C,1,1)
    返回：物理单位下的张量。
    """
    while mean_c11.ndim < norm_tensor.ndim:
        mean_c11 = mean_c11.unsqueeze(0)  # 在前面补 batch/time 维度
        std_c11  = std_c11.unsqueeze(0)
    return norm_tensor * std_c11 + mean_c11


# =========================
# 1. 构建模型 & 加载 checkpoint
# =========================

def build_model_for_eval(cfg, n_channels: int, device: torch.device):
    """
    用 TrainConfig 和通道数 C 来 new 一个 ConvSeq2Seq。
    不依赖你之前的 _build_model。
    """
    hid_enc = [cfg.hidden_dim] * cfg.n_enc_layers
    hid_dec = [cfg.hidden_dim] * cfg.n_dec_layers

    model = ConvSeq2Seq(
        input_dim=n_channels,
        enc_cell_type=cfg.enc_cell_type,
        dec_cell_type=cfg.dec_cell_type,
        hidden_dims_enc=hid_enc,
        hidden_dims_dec=hid_dec,
        kernel_size=cfg.kernel_size,
        layernorm=cfg.layernorm,
        peephole=cfg.peephole,
        unet_head=cfg.unet_head,
    ).to(device)

    return model


def load_model_from_checkpoint(ckpt_path: str, cfg, n_channels: int, device: torch.device):
    """
    构建好模型并从 checkpoint 加载权重。
    """
    model = build_model_for_eval(cfg, n_channels, device)
    state = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(state)
    model.to(device)
    model.eval()
    print(f"[INFO] Loaded checkpoint: {ckpt_path}")
    return model


# =========================
# 2. lead-time RMSE（单变量）
# =========================

def compute_lead_rmse_curve_for_var(
    model: torch.nn.Module,
    data_loader,
    mean_c11: torch.Tensor,
    std_c11: torch.Tensor,
    var_idx: int,
    time_step_hours: int,
    max_lead_hours: int,
    device: torch.device,
    precip_idx: int | None = None,
    precip_is_log1p_mm: bool = True,
):
    """
    对某个变量计算「纬向加权 RMSE vs lead time」曲线（物理单位）。

    - data_loader 输出 (x_in, y_out_norm)，标准化空间，形状 (B, Tin/Tout, C, H, W)
    - 用 mean_c11/std_c11 反归一化回物理单位
    - 如果 precip_idx 不为 None 且 precip_is_log1p_mm=True，则该通道从 log1p(mm) -> mm
    - 真正能算到的最大 lead 步数 = min(y_len, max_lead_hours / time_step_hours)
    """
    model.eval()
    mean_c11 = mean_c11.to(device)
    std_c11  = std_c11.to(device)

    # 探测 y_len
    with torch.no_grad():
        x0, y0 = next(iter(data_loader))
        Tout = y0.shape[1]

    max_steps = min(Tout, max_lead_hours // time_step_hours)
    if max_steps <= 0:
        raise ValueError("max_lead_hours 太小或者 y_len 太短，导致没有可用的 lead 步数。")

    rmse_sum = torch.zeros(max_steps, device=device)
    n_samp   = torch.zeros(max_steps, device=device)

    with torch.no_grad():
        for x_in, y_out_norm in data_loader:
            x_in = x_in.to(device)
            y_out_norm = y_out_norm.to(device)
            B = x_in.size(0)

            # 自由解码
            pred_norm = model(
                x_in,
                y_len=y_out_norm.shape[1],
                teacher_forcing_ratio=0.0,
                targets=None,
            )  # (B, Tout, C, H, W)

            # 反归一化到物理单位（所有变量）
            pred = denorm_all_channels(pred_norm, mean_c11, std_c11)  # (B,T,C,H,W)
            y    = denorm_all_channels(y_out_norm, mean_c11, std_c11)

            # 降水通道：log1p(mm) -> mm
            if precip_idx is not None and precip_is_log1p_mm:
                tp_pred_log = pred[:, :, precip_idx, :, :]
                tp_y_log    = y[:, :, precip_idx, :, :]

                tp_pred_mm = torch.expm1(tp_pred_log).clamp(min=0.0)
                tp_y_mm    = torch.expm1(tp_y_log).clamp(min=0.0)

                pred[:, :, precip_idx, :, :] = tp_pred_mm
                y[:,   :, precip_idx, :, :] = tp_y_mm

            # 每个 lead t 算 RMSE
            for t in range(max_steps):
                pred_t = pred[:, t, :, :, :]   # (B,C,H,W)
                y_t    = y[:,   t, :, :, :]    # (B,C,H,W)

                rmse_bc = weighted_rmse_channels(pred_t, y_t)  # (B,C)
                rmse_b  = rmse_bc[:, var_idx]                  # (B,)

                rmse_sum[t] += rmse_b.sum()
                n_samp[t]   += B

    rmse_lead  = (rmse_sum / n_samp).cpu().numpy()              # (max_steps,)
    lead_hours = np.arange(1, max_steps + 1) * time_step_hours  # [Δt, 2Δt, ..., max_lead]

    return lead_hours, rmse_lead


def save_lead_rmse_npz(
    out_path: str,
    lead_hours: np.ndarray,
    rmse_lead: np.ndarray,
    var_name: str,
    ckpt_path: str,
):
    """
    把 lead-time RMSE 曲线存成 npz，方便以后复用。
    """
    out_path = Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        out_path,
        lead_hours=lead_hours,
        rmse_lead=rmse_lead,
        var_name=np.array(var_name),
        ckpt_path=np.array(ckpt_path),
    )
    print(f"[INFO] Saved lead RMSE curve to: {out_path}")


# =========================
# 3. 全局 metrics（表格用）
# =========================

def compute_global_metrics_for_all_vars(
    model: torch.nn.Module,
    data_loader,
    mean_c11: torch.Tensor,
    std_c11: torch.Tensor,
    device: torch.device,
    precip_idx: int | None = None,
    precip_is_log1p_mm: bool = True,
):
    """
    返回每个变量的 global RMSE/MAE/ACC/RMSB（物理单位下）。
    使用你提供的 weighted_rmse_channels / weighted_mae_channels /
    weighted_acc_channels / weighted_rmsb_channels。
    """
    model.eval()
    mean_c11 = mean_c11.to(device)
    std_c11  = std_c11.to(device)

    preds = []
    trues = []

    with torch.no_grad():
        for x_in, y_out_norm in data_loader:
            x_in = x_in.to(device)
            y_out_norm = y_out_norm.to(device)

            pred_norm = model(
                x_in,
                y_len=y_out_norm.shape[1],
                teacher_forcing_ratio=0.0,
                targets=None,
            )

            pred = denorm_all_channels(pred_norm, mean_c11, std_c11)  # (B,T,C,H,W)
            y    = denorm_all_channels(y_out_norm, mean_c11, std_c11)

            if precip_idx is not None and precip_is_log1p_mm:
                tp_pred_log = pred[:, :, precip_idx, :, :]
                tp_y_log    = y[:, :, precip_idx, :, :]

                tp_pred_mm = torch.expm1(tp_pred_log).clamp(min=0.0)
                tp_y_mm    = torch.expm1(tp_y_log).clamp(min=0.0)

                pred[:, :, precip_idx, :, :] = tp_pred_mm
                y[:,   :, precip_idx, :, :] = tp_y_mm

            B, T, Cc, H, W = pred.shape
            preds.append(pred.view(B * T, Cc, H, W))
            trues.append(y.view(B * T, Cc, H, W))

    pred_all = torch.cat(preds, dim=0)  # (N, C, H, W)
    true_all = torch.cat(trues, dim=0)  # (N, C, H, W)

    # 这些函数都预期 [B,C,H,W]
    rmse_bt = weighted_rmse_channels(pred_all, true_all)  # (N,C)
    mae_bt  = weighted_mae_channels(pred_all, true_all)   # (N,C)
    acc_bt  = weighted_acc_channels(pred_all, true_all)   # (N,C)

    rmse_global = rmse_bt.mean(dim=0).cpu().numpy()  # (C,)
    mae_global  = mae_bt.mean(dim=0).cpu().numpy()   # (C,)
    acc_global  = acc_bt.mean(dim=0).cpu().numpy()   # (C,)

    rmsb_global = weighted_rmsb_channels(pred_all, true_all).cpu().numpy()  # (C,)

    return {
        "rmse": rmse_global,
        "mae":  mae_global,
        "acc":  acc_global,
        "rmsb": rmsb_global,
    }


In [ ]:
import json
from pathlib import Path
import torch

def load_model_from_checkpoint_auto_cfg(
    ckpt_path: str,
    C: int,
    device: torch.device,
) -> tuple[ConvSeq2Seq, TrainConfig]:
    """
    从 checkpoint 路径自动：
      1) 找到 run_dir = ckpt_path/../..
      2) 读取 run_dir/config.json -> TrainConfig
      3) 用 cfg + C 构建模型
      4) 加载 state_dict

    返回: (model, cfg)
    """
    ckpt_path = Path(ckpt_path)
    run_dir = ckpt_path.parents[1]              # .../enc_xxx/
    cfg_path = run_dir / "config.json"

    with open(cfg_path, "r", encoding="utf-8") as f:
        cfg_dict = json.load(f)

    cfg = TrainConfig(**cfg_dict)

    # 根据 C 和 cfg 构建模型
    model = _build_model(cfg, C, device)        # 用你之前的 _build_model
    state = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(state)
    model.eval()

    print(f"[INFO] Loaded checkpoint: {ckpt_path}")
    print(f"[INFO] Loaded config from: {cfg_path}")

    return model, cfg



import os
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

VAR_UNITS = {
    "2m_temperature": "K",
    "10m_u_component_of_wind": "m/s",
    "10m_v_component_of_wind": "m/s",
    "mean_sea_level_pressure": "Pa",
    "total_precipitation": "mm/6h",      # 你现在的设定：6h 累积 mm
    "total_precipitation_6hr": "mm/6h",  # 防止你变量名用这个
}


def get_mean_std_c11(ds_train):
    """
    ds_train.mean / std: (C,1,1) or (C,1,1,1)? 按你之前的写法：
        ds_train.mean: (C, 1, 1)
    你现在是 (C,1,1,1)，下面稍微保险点做一下兼容。
    """
    mean = ds_train.mean  # (C,1,1,1)
    std  = ds_train.std   # (C,1,1,1)
    # 变成 (1,C,1,1)
    if mean.ndim == 4 and mean.shape[0] == 1:
        mean_c11 = mean
        std_c11  = std
    elif mean.ndim == 4 and mean.shape[1] == 1:
        # (C,1,1,1) -> (1,C,1,1)
        mean_c11 = mean.permute(1,0,2,3).contiguous()
        std_c11  = std.permute(1,0,2,3).contiguous()
    else:
        raise ValueError(f"Unexpected mean/std shape: {mean.shape}")
    return mean_c11, std_c11


def evaluate_checkpoint_and_save_all_metrics(
    ds_train,
    test_loader,
    variables,
    ckpt_path: str,
    # 可选：专门用于 lead-time（比如 y_len=8）
    test_loader_lead=None,
    time_step_hours: int = 6,
    max_lead_hours: int = 48,
    precip_var_name: str = "total_precipitation",
    precip_is_log1p_mm: bool = True,
    show_plots: bool = False,
    # 可视化参数
    num_vis_samples: int = 4,
    viz_use_last_t: bool = True,
    viz_channel: int | None = None,
    viz_all_channels: bool = False,
):
    """
    对单个 checkpoint 做完整评估（使用 config.json 自动加载 config）：

      - 从 ds_train 读取 mean/std
      - 用 mean/std 推出 C，并构建+加载模型
      - 对每个变量：计算 lead-time RMSE (用 test_loader_lead 或 test_loader)
      - 在 test_loader 上计算全局 RMSE/MAE/ACC/RMSB
      - 在 test_loader 上画若干可视化样本

    所有输出放在:
      run_dir = ckpt_path/../..
      metrics_dir = run_dir / "metrics"
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ---------- 路径 & 目录 ----------
    ckpt_path = Path(ckpt_path)
    run_dir = ckpt_path.parents[1]
    metrics_dir = run_dir / "metrics"
    metrics_dir.mkdir(parents=True, exist_ok=True)
    print(f"[INFO] metrics will be saved to: {metrics_dir}")

    # ---------- 1) mean/std & 通道数 C ----------
    mean_c11, std_c11 = get_mean_std_c11(ds_train)
    print("mean_c11:", mean_c11.shape, "std_c11:", std_c11.shape)
    C = mean_c11.shape[1]
    print(f"[INFO] inferred C from mean/std: C={C}")

    # ---------- 2) 自动从 config.json + ckpt 恢复模型 ----------
    model, cfg = load_model_from_checkpoint_auto_cfg(
        ckpt_path=str(ckpt_path),
        C=C,
        device=device,
    )

    # ---------- 3) 降水 index ----------
    precip_idx = None
    if precip_var_name in variables:
        precip_idx = variables.index(precip_var_name)
        print(f"[INFO] Precipitation variable: '{precip_var_name}' at index {precip_idx}")
    else:
        print(f"[INFO] Precipitation variable '{precip_var_name}' not found; "
              f"precip-specific postprocessing will be skipped.")
        precip_is_log1p_mm = False

    # ---------- 4) 每个变量的 lead-time RMSE ----------
    dl_for_lead = test_loader_lead if test_loader_lead is not None else test_loader
    per_var_lead = {}

    for var_name in variables:
        var_idx = variables.index(var_name)
        print(f"\n[Lead RMSE] Evaluating variable: {var_name} (idx={var_idx})")

        lead_hours, rmse_lead = compute_lead_rmse_curve_for_var(
            model=model,
            data_loader=dl_for_lead,
            mean_c11=mean_c11,
            std_c11=std_c11,
            var_idx=var_idx,
            time_step_hours=time_step_hours,
            max_lead_hours=max_lead_hours,
            device=device,
            precip_idx=precip_idx,
            precip_is_log1p_mm=precip_is_log1p_mm,
        )

        print("  lead_hours:", lead_hours)
        print("  rmse_lead:", rmse_lead)

        per_var_lead[var_name] = (lead_hours, rmse_lead)

        # 保存 npz
        out_npz = metrics_dir / f"lead_rmse_{var_name}_up_to_{max_lead_hours}h.npz"
        save_lead_rmse_npz(str(out_npz), lead_hours, rmse_lead, var_name, str(ckpt_path))

        # 画图
        unit = VAR_UNITS.get(var_name, "")
        ylabel = f"Weighted RMSE ({unit})" if unit else "Weighted RMSE"

        plt.figure(figsize=(5, 4))
        plt.plot(lead_hours / 24.0, rmse_lead, marker="o")
        plt.xlabel("Lead time (days)")
        plt.ylabel(ylabel)
        plt.title(f"{var_name} – RMSE vs Lead Time (Test)")
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.tight_layout()

        out_png = metrics_dir / f"lead_rmse_{var_name}_up_to_{max_lead_hours}h.png"
        plt.savefig(out_png, dpi=160)
        print(f"  [saved figure] {out_png}")

        if show_plots:
            plt.show()
        else:
            plt.close()

    # ---------- 5) 全局 metrics（在 test_loader 上，通常 y_len=4） ----------
    print("\n[Global metrics per variable on test set]")
    metrics_all = compute_global_metrics_for_all_vars(
        model=model,
        data_loader=test_loader,
        mean_c11=mean_c11,
        std_c11=std_c11,
        device=device,
        precip_idx=precip_idx,
        precip_is_log1p_mm=precip_is_log1p_mm,
    )

    for c, vname in enumerate(variables):
        unit = VAR_UNITS.get(vname, "")
        unit_str = f" ({unit})" if unit else ""
        print(
            f"{vname:30s}{unit_str:12s} | "
            f"RMSE={metrics_all['rmse'][c]:.4g}  "
            f"MAE={metrics_all['mae'][c]:.4g}  "
            f"ACC={metrics_all['acc'][c]:.4f}  "
            f"RMSB={metrics_all['rmsb'][c]:.4g}"
        )

    global_npz = metrics_dir / "global_metrics_all_vars.npz"
    np.savez_compressed(
        global_npz,
        variables=np.array(variables),
        rmse=metrics_all["rmse"],
        mae=metrics_all["mae"],
        acc=metrics_all["acc"],
        rmsb=metrics_all["rmsb"],
        ckpt_path=np.array(str(ckpt_path)),
    )
    print(f"\n[INFO] Saved global metrics to: {global_npz}")

    # ---------- 6) 可视化若干 test 样本 ----------
    vis_dir = metrics_dir / "vis"
    vis_dir.mkdir(parents=True, exist_ok=True)
    print(f"[INFO] Saving visualization samples to: {vis_dir}")

    pin = torch.cuda.is_available()
    vis_loader = DataLoader(
        test_loader.dataset,
        batch_size=1,
        shuffle=True,
        num_workers=0,
        pin_memory=pin,
    )

    model.eval()
    with torch.no_grad():
        count = 0
        for x_in, y_out in vis_loader:
            x_in = x_in.to(device, non_blocking=True)
            y_out = y_out.to(device, non_blocking=True)

            save_prefix = vis_dir / f"test_sample_{count:03d}"
            visualize_once(
                model=model,
                x_in=x_in,
                y_out=y_out,
                mean=mean_c11.to(device),
                std=std_c11.to(device),
                save_prefix=save_prefix,
                use_last_t=viz_use_last_t,
                viz_channel=viz_channel,
                draw_all_channels=viz_all_channels,
            )
            print(f"  [vis] saved {save_prefix}.png")

            count += 1
            if count >= num_vis_samples:
                break

    return model, metrics_all, per_var_lead


#### convgru+unet

In [ ]:
# batch_size = 8

# train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=True,  num_workers=4, pin_memory=True)
# val_loader   = DataLoader(ds_val,   batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# mean_c11 = ds_train.mean.permute(1, 0, 2, 3).contiguous()  # (1,C,1,1)
# std_c11  = ds_train.std.permute(1, 0, 2, 3).contiguous()   # (1,C,1,1)

mean_c11 = ds_train.mean   # (1,C,1,1)
std_c11  = ds_train.std    # (1,C,1,1)

convgru_unet_cfg = TrainConfig(
    batch_size=batch_size,
    lr=1e-4,
    max_epochs=50,
    enc_cell_type="convgru",
    dec_cell_type="convgru",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=False,
    unet_head=True,
    out_root="./6hr_runs_w_unet",
    viz_every_n_epochs=10,
    viz_use_last_t=True,
    viz_channel=None,
    save_all_channels_png=True,
)

convgru_unet_model, convgru_unet_history = train(convgru_unet_cfg, train_loader, val_loader, mean_c11, std_c11)

[Epoch 1/50] TF=1.00 | train MSE=0.238816 RMSE=0.488688 MAE=0.287056 | val MSE=0.276706 RMSE=0.526028 MAE=0.297245
  >> Saved best model (val MSE=0.276706)
[Epoch 2/50] TF=1.00 | train MSE=0.119056 RMSE=0.345045 MAE=0.195205 | val MSE=0.252701 RMSE=0.502694 MAE=0.275095
  >> Saved best model (val MSE=0.252701)
[Epoch 3/50] TF=1.00 | train MSE=0.106208 RMSE=0.325896 MAE=0.181229 | val MSE=0.228772 RMSE=0.478301 MAE=0.256214
  >> Saved best model (val MSE=0.228772)
[Epoch 4/50] TF=1.00 | train MSE=0.098800 RMSE=0.314324 MAE=0.172879 | val MSE=0.225265 RMSE=0.474620 MAE=0.256235
  >> Saved best model (val MSE=0.225265)
[Epoch 5/50] TF=1.00 | train MSE=0.093532 RMSE=0.305829 MAE=0.166710 | val MSE=0.216599 RMSE=0.465402 MAE=0.246261
  >> Saved best model (val MSE=0.216599)
[Epoch 6/50] TF=1.00 | train MSE=0.089842 RMSE=0.299736 MAE=0.162112 | val MSE=0.205940 RMSE=0.453807 MAE=0.238630
  >> Saved best model (val MSE=0.205940)
[Epoch 7/50] TF=1.00 | train MSE=0.086444 RMSE=0.294014 MAE=0.15

In [ ]:

ds_test_lead = ERA5XYDataset6h(
    np_test,
    variables,
    x_len=8,
    y_len=8,   # ← 48h lead
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
    precip_to_mm=True,
    precip_log1p=True,
)

test_loader_lead = DataLoader(ds_test_lead, batch_size=batch_size, shuffle=False, num_workers=0)


ckpt_path = "./6hr_runs_w_unet/enc_convgru_dec_convgru_20251211-201151/checkpoints/best.pt"

model, metrics_all, per_var_lead = evaluate_checkpoint_and_save_all_metrics(
    ds_train=ds_train,
    test_loader=test_loader,
    variables=variables,
    ckpt_path=ckpt_path,
    test_loader_lead=test_loader_lead,
    time_step_hours=6,
    max_lead_hours=48,
    precip_var_name="total_precipitation_6hr",  # precip variable name
    precip_is_log1p_mm=True,
    show_plots=True,
    num_vis_samples=4,
    viz_use_last_t=True,
    viz_channel=None,
    viz_all_channels=True,
)

[INFO] metrics will be saved to: 6hr_runs_w_unet/enc_convgru_dec_convgru_20251211-201151/metrics
mean_c11: torch.Size([1, 5, 1, 1]) std_c11: torch.Size([1, 5, 1, 1])
[INFO] inferred C from mean/std: C=5
[INFO] Loaded checkpoint: 6hr_runs_w_unet/enc_convgru_dec_convgru_20251211-201151/checkpoints/best.pt
[INFO] Loaded config from: 6hr_runs_w_unet/enc_convgru_dec_convgru_20251211-201151/config.json
[INFO] Precipitation variable: 'total_precipitation_6hr' at index 4

[Lead RMSE] Evaluating variable: 2m_temperature (idx=0)
  lead_hours: [ 6 12 18 24 30 36 42 48]
  rmse_lead: [3.0689409 3.4653127 4.102208  4.503538  5.014202  5.5162764 5.9194713
 6.244851 ]
[INFO] Saved lead RMSE curve to: 6hr_runs_w_unet/enc_convgru_dec_convgru_20251211-201151/metrics/lead_rmse_2m_temperature_up_to_48h.npz
  [saved figure] 6hr_runs_w_unet/enc_convgru_dec_convgru_20251211-201151/metrics/lead_rmse_2m_temperature_up_to_48h.png

[Lead RMSE] Evaluating variable: 10m_u_component_of_wind (idx=1)
  lead_hours: [ 6

#### convrnn+unet

In [ ]:

mean_c11 = ds_train.mean   # (1,C,1,1)
std_c11  = ds_train.std    # (1,C,1,1)

convrnn_unet_cfg = TrainConfig(
    batch_size=batch_size,
    lr=1e-4,
    max_epochs=50,
    enc_cell_type="convrnn",
    dec_cell_type="convrnn",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=False,
    unet_head=True,
    out_root="./6hr_runs_w_unet",
    viz_every_n_epochs=10,
    viz_use_last_t=True,
    viz_channel=None,
    save_all_channels_png=True,
)

convrnn_unet_model, convrnn_unet_history = train(convrnn_unet_cfg, train_loader, val_loader, mean_c11, std_c11)

[Epoch 1/50] TF=1.00 | train MSE=0.281140 RMSE=0.530227 MAE=0.312187 | val MSE=0.291788 RMSE=0.540174 MAE=0.309980
  >> Saved best model (val MSE=0.291788)
[Epoch 2/50] TF=1.00 | train MSE=0.119460 RMSE=0.345629 MAE=0.195249 | val MSE=0.255836 RMSE=0.505802 MAE=0.273868
  >> Saved best model (val MSE=0.255836)
[Epoch 3/50] TF=1.00 | train MSE=0.106599 RMSE=0.326495 MAE=0.181494 | val MSE=0.247663 RMSE=0.497658 MAE=0.273260
  >> Saved best model (val MSE=0.247663)
[Epoch 4/50] TF=1.00 | train MSE=0.099382 RMSE=0.315250 MAE=0.173570 | val MSE=0.225322 RMSE=0.474681 MAE=0.251960
  >> Saved best model (val MSE=0.225322)
[Epoch 5/50] TF=1.00 | train MSE=0.094478 RMSE=0.307372 MAE=0.167790 | val MSE=0.219990 RMSE=0.469031 MAE=0.249418
  >> Saved best model (val MSE=0.219990)
[Epoch 6/50] TF=1.00 | train MSE=0.090704 RMSE=0.301172 MAE=0.163401 | val MSE=0.213486 RMSE=0.462045 MAE=0.244796
  >> Saved best model (val MSE=0.213486)
[Epoch 7/50] TF=1.00 | train MSE=0.087887 RMSE=0.296458 MAE=0.15

In [ ]:

ds_test_lead = ERA5XYDataset6h(
    np_test,
    variables,
    x_len=8,
    y_len=8,   # ← 48h lead
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
    precip_to_mm=True,
    precip_log1p=True,
)

test_loader_lead = DataLoader(ds_test_lead, batch_size=batch_size, shuffle=False, num_workers=0)


ckpt_path = "./6hr_runs_w_unet/enc_convrnn_dec_convrnn_20251211-224107/checkpoints/best.pt"

model, metrics_all, per_var_lead = evaluate_checkpoint_and_save_all_metrics(
    ds_train=ds_train,
    test_loader=test_loader,           # y_len=4
    variables=variables,
    ckpt_path=ckpt_path,
    test_loader_lead=test_loader_lead, # y_len=8, 专门用来画 lead curve 到 48h
    time_step_hours=6,
    max_lead_hours=48,
    precip_var_name="total_precipitation_6hr",  # precip variable name
    precip_is_log1p_mm=True,
    show_plots=True,
    num_vis_samples=4,
    viz_use_last_t=True,
    viz_channel=None,
    viz_all_channels=True,
)

[INFO] metrics will be saved to: 6hr_runs_w_unet/enc_convrnn_dec_convrnn_20251211-224107/metrics
mean_c11: torch.Size([1, 5, 1, 1]) std_c11: torch.Size([1, 5, 1, 1])
[INFO] inferred C from mean/std: C=5
[INFO] Loaded checkpoint: 6hr_runs_w_unet/enc_convrnn_dec_convrnn_20251211-224107/checkpoints/best.pt
[INFO] Loaded config from: 6hr_runs_w_unet/enc_convrnn_dec_convrnn_20251211-224107/config.json
[INFO] Precipitation variable: 'total_precipitation_6hr' at index 4

[Lead RMSE] Evaluating variable: 2m_temperature (idx=0)
  lead_hours: [ 6 12 18 24 30 36 42 48]
  rmse_lead: [ 2.8656409  3.618819   4.9545426  6.397257   8.179749  10.033303
 11.980595  14.096959 ]
[INFO] Saved lead RMSE curve to: 6hr_runs_w_unet/enc_convrnn_dec_convrnn_20251211-224107/metrics/lead_rmse_2m_temperature_up_to_48h.npz
  [saved figure] 6hr_runs_w_unet/enc_convrnn_dec_convrnn_20251211-224107/metrics/lead_rmse_2m_temperature_up_to_48h.png

[Lead RMSE] Evaluating variable: 10m_u_component_of_wind (idx=1)
  lead_hou

#### convlstm+unet

In [ ]:

mean_c11 = ds_train.mean   # (1,C,1,1)
std_c11  = ds_train.std    # (1,C,1,1)

convlstm_unet_cfg = TrainConfig(
    batch_size=batch_size,
    lr=1e-4,
    max_epochs=50,
    enc_cell_type="convlstm",
    dec_cell_type="convlstm",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=True,
    unet_head=True,
    out_root="./6hr_runs_w_unet",
    viz_every_n_epochs=10,
    viz_use_last_t=True,
    viz_channel=None,
    save_all_channels_png=True,
)

convlstm_unet_model, convlstm_unet_history = train(convlstm_unet_cfg, train_loader, val_loader, mean_c11, std_c11)

[Epoch 1/50] TF=1.00 | train MSE=0.227249 RMSE=0.476707 MAE=0.274590 | val MSE=0.286049 RMSE=0.534835 MAE=0.298742
  >> Saved best model (val MSE=0.286049)
[Epoch 2/50] TF=1.00 | train MSE=0.119544 RMSE=0.345751 MAE=0.195258 | val MSE=0.248512 RMSE=0.498510 MAE=0.272848
  >> Saved best model (val MSE=0.248512)
[Epoch 3/50] TF=1.00 | train MSE=0.108063 RMSE=0.328729 MAE=0.183283 | val MSE=0.234223 RMSE=0.483966 MAE=0.264053
  >> Saved best model (val MSE=0.234223)
[Epoch 4/50] TF=1.00 | train MSE=0.100857 RMSE=0.317580 MAE=0.175436 | val MSE=0.224128 RMSE=0.473421 MAE=0.260847
  >> Saved best model (val MSE=0.224128)
[Epoch 5/50] TF=1.00 | train MSE=0.095586 RMSE=0.309171 MAE=0.169130 | val MSE=0.221233 RMSE=0.470354 MAE=0.250080
  >> Saved best model (val MSE=0.221233)
[Epoch 6/50] TF=1.00 | train MSE=0.090663 RMSE=0.301104 MAE=0.162876 | val MSE=0.209751 RMSE=0.457986 MAE=0.239560
  >> Saved best model (val MSE=0.209751)
[Epoch 7/50] TF=1.00 | train MSE=0.087529 RMSE=0.295853 MAE=0.15

In [ ]:

ds_test_lead = ERA5XYDataset6h(
    np_test,
    variables,
    x_len=8,
    y_len=8,
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
    precip_to_mm=True,
    precip_log1p=True,
)

test_loader_lead = DataLoader(ds_test_lead, batch_size=batch_size, shuffle=False, num_workers=0)


ckpt_path = "./6hr_runs_w_unet/enc_convlstm_dec_convlstm_20251211-235905/checkpoints/best.pt"

model, metrics_all, per_var_lead = evaluate_checkpoint_and_save_all_metrics(
    ds_train=ds_train,
    test_loader=test_loader,
    variables=variables,
    ckpt_path=ckpt_path,
    test_loader_lead=test_loader_lead,
    time_step_hours=6,
    max_lead_hours=48,
    precip_var_name="total_precipitation_6hr",  # precip variable name
    precip_is_log1p_mm=True,
    show_plots=True,
    num_vis_samples=4,
    viz_use_last_t=True,
    viz_channel=None,
    viz_all_channels=True,
)

[INFO] metrics will be saved to: 6hr_runs_w_unet/enc_convlstm_dec_convlstm_20251211-235905/metrics
mean_c11: torch.Size([1, 5, 1, 1]) std_c11: torch.Size([1, 5, 1, 1])
[INFO] inferred C from mean/std: C=5
[INFO] Loaded checkpoint: 6hr_runs_w_unet/enc_convlstm_dec_convlstm_20251211-235905/checkpoints/best.pt
[INFO] Loaded config from: 6hr_runs_w_unet/enc_convlstm_dec_convlstm_20251211-235905/config.json
[INFO] Precipitation variable: 'total_precipitation_6hr' at index 4

[Lead RMSE] Evaluating variable: 2m_temperature (idx=0)
  lead_hours: [ 6 12 18 24 30 36 42 48]
  rmse_lead: [3.3467786 3.442735  3.6127858 3.8097866 4.1958666 4.5095086 4.7991867
 5.1101294]
[INFO] Saved lead RMSE curve to: 6hr_runs_w_unet/enc_convlstm_dec_convlstm_20251211-235905/metrics/lead_rmse_2m_temperature_up_to_48h.npz
  [saved figure] 6hr_runs_w_unet/enc_convlstm_dec_convlstm_20251211-235905/metrics/lead_rmse_2m_temperature_up_to_48h.png

[Lead RMSE] Evaluating variable: 10m_u_component_of_wind (idx=1)
  lead_

#### convlstm

In [ ]:

mean_c11 = ds_train.mean   # (1,C,1,1)
std_c11  = ds_train.std    # (1,C,1,1)

convlstm_cfg = TrainConfig(
    batch_size=batch_size,
    lr=1e-4,
    max_epochs=50,
    enc_cell_type="convlstm",
    dec_cell_type="convlstm",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=True,
    unet_head=True,
    out_root="./6hr_runs_wo_unet",
    viz_every_n_epochs=10,
    viz_use_last_t=True,
    viz_channel=None,
    save_all_channels_png=True,
)

convlstm_model, convlstm_history = train(convlstm_cfg, train_loader, val_loader, mean_c11, std_c11)

[Epoch 1/50] TF=1.00 | train MSE=0.228475 RMSE=0.477990 MAE=0.281111 | val MSE=0.314984 RMSE=0.561235 MAE=0.324257
  >> Saved best model (val MSE=0.314984)
[Epoch 2/50] TF=1.00 | train MSE=0.117849 RMSE=0.343292 MAE=0.194564 | val MSE=0.256682 RMSE=0.506638 MAE=0.276039
  >> Saved best model (val MSE=0.256682)
[Epoch 3/50] TF=1.00 | train MSE=0.105678 RMSE=0.325082 MAE=0.181489 | val MSE=0.224008 RMSE=0.473295 MAE=0.257198
  >> Saved best model (val MSE=0.224008)
[Epoch 4/50] TF=1.00 | train MSE=0.098340 RMSE=0.313591 MAE=0.172938 | val MSE=0.235235 RMSE=0.485010 MAE=0.265437
[Epoch 5/50] TF=1.00 | train MSE=0.093363 RMSE=0.305553 MAE=0.167033 | val MSE=0.212260 RMSE=0.460717 MAE=0.243675
  >> Saved best model (val MSE=0.212260)
[Epoch 6/50] TF=1.00 | train MSE=0.089316 RMSE=0.298858 MAE=0.161940 | val MSE=0.205542 RMSE=0.453367 MAE=0.238440
  >> Saved best model (val MSE=0.205542)
[Epoch 7/50] TF=1.00 | train MSE=0.086304 RMSE=0.293775 MAE=0.158283 | val MSE=0.204193 RMSE=0.451877 MAE

In [ ]:

ds_test_lead = ERA5XYDataset6h(
    np_test,
    variables,
    x_len=8,
    y_len=8,   # ← 48h lead
    stride=1,
    normalize=True,
    mean=ds_train.mean,
    std=ds_train.std,
    precip_to_mm=True,
    precip_log1p=True,
)

test_loader_lead = DataLoader(ds_test_lead, batch_size=batch_size, shuffle=False, num_workers=0)


ckpt_path = "./6hr_runs_wo_unet/enc_convlstm_dec_convlstm_20251212-013104/checkpoints/best.pt"

model, metrics_all, per_var_lead = evaluate_checkpoint_and_save_all_metrics(
    ds_train=ds_train,
    test_loader=test_loader,
    variables=variables,
    ckpt_path=ckpt_path,
    test_loader_lead=test_loader_lead,
    time_step_hours=6,
    max_lead_hours=48,
    precip_var_name="total_precipitation_6hr",
    precip_is_log1p_mm=True,
    show_plots=True,
    num_vis_samples=4,
    viz_use_last_t=True,
    viz_channel=None,
    viz_all_channels=True,
)

[INFO] metrics will be saved to: 6hr_runs_wo_unet/enc_convlstm_dec_convlstm_20251212-013104/metrics
mean_c11: torch.Size([1, 5, 1, 1]) std_c11: torch.Size([1, 5, 1, 1])
[INFO] inferred C from mean/std: C=5
[INFO] Loaded checkpoint: 6hr_runs_wo_unet/enc_convlstm_dec_convlstm_20251212-013104/checkpoints/best.pt
[INFO] Loaded config from: 6hr_runs_wo_unet/enc_convlstm_dec_convlstm_20251212-013104/config.json
[INFO] Precipitation variable: 'total_precipitation_6hr' at index 4

[Lead RMSE] Evaluating variable: 2m_temperature (idx=0)
  lead_hours: [ 6 12 18 24 30 36 42 48]
  rmse_lead: [3.039699  3.5818639 4.0992017 4.5291452 5.1147766 5.53357   6.025065
 6.653648 ]
[INFO] Saved lead RMSE curve to: 6hr_runs_wo_unet/enc_convlstm_dec_convlstm_20251212-013104/metrics/lead_rmse_2m_temperature_up_to_48h.npz
  [saved figure] 6hr_runs_wo_unet/enc_convlstm_dec_convlstm_20251212-013104/metrics/lead_rmse_2m_temperature_up_to_48h.png

[Lead RMSE] Evaluating variable: 10m_u_component_of_wind (idx=1)
  l

### dummy

In [ ]:
convgru_unet_cfg = TrainConfig(
    data_dir="./",
    max_epochs=100,
    batch_size=2,
    enc_cell_type="convgru",
    dec_cell_type="convgru",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=False,
    unet_head=True,
    out_root="./runs_w_unet",
    viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
    viz_use_last_t=True,       # 可切换 False 以随机时间步
    viz_channel=None,          # None=随机通道；也可设 0/1/2...
    save_all_channels_png=True
)
convgru_unet_model, convgru_unet_history = train(convgru_unet_cfg)


数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.175221 RMSE=0.418595 MAE=0.273519 | val MSE=0.305383 RMSE=0.552615 MAE=0.358018
  >> Saved best model (val MSE=0.305383)
[Epoch 2/100] TF=1.00 | train MSE=0.046581 RMSE=0.215827 MAE=0.146971 | val MSE=0.306194 RMSE=0.553348 MAE=0.352155
[Epoch 3/100] TF=1.00 | train MSE=0.035252 RMSE=0.187756 MAE=0.126996 | val MSE=0.247547 RMSE=0.497541 MAE=0.321898
  >> Saved best model (val MSE=0.247547)
[Epoch 4/100] TF=1.00 | train MSE=0.030717 RMSE=0.175263 MAE=0.118487 | val MSE=0.289808 RMSE=0.538338 MAE=0.342817
[Epoch 5/100] TF=1.00 | train MSE=0.028681 RMSE=0.169354 MAE=0.114073 | val MSE=0.255584 RMSE=0.505553 MAE=0.320323
[Epoch 6/100] TF=1.00 | train MSE=0.026178 RMSE=0.161796 MAE=0.107988 | val MSE=0.202908 RMSE=0.450453 MAE=0.283811
  >> Saved best model (val MSE=0.202908)
[Epoch 7/100] TF=1.00 | train MSE=0.024642 RMSE=0.156978 MAE=0.104406 | val MSE=0.193176 RMSE=0.439518 MAE=0.277407
  >> Saved best model (val MSE=0.1

##### convrnn+unet

In [ ]:
convrnn_unet_cfg = TrainConfig(
    data_dir="./",
    max_epochs=100,
    batch_size=2,
    enc_cell_type="convrnn",
    dec_cell_type="convrnn",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=False,
    unet_head=True,
    out_root="./runs_w_unet",
    viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
    viz_use_last_t=True,       # 可切换 False 以随机时间步
    viz_channel=None,          # None=随机通道；也可设 0/1/2...
    save_all_channels_png=True
)
convrnn_unet__model, convrnn_unet_history = train(convrnn_unet_cfg)


数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.198879 RMSE=0.445959 MAE=0.293874 | val MSE=0.309009 RMSE=0.555886 MAE=0.374112
  >> Saved best model (val MSE=0.309009)
[Epoch 2/100] TF=1.00 | train MSE=0.052416 RMSE=0.228946 MAE=0.156258 | val MSE=0.314910 RMSE=0.561168 MAE=0.378718
[Epoch 3/100] TF=1.00 | train MSE=0.041373 RMSE=0.203405 MAE=0.138453 | val MSE=0.275440 RMSE=0.524824 MAE=0.348831
  >> Saved best model (val MSE=0.275440)
[Epoch 4/100] TF=1.00 | train MSE=0.035075 RMSE=0.187284 MAE=0.127586 | val MSE=0.294691 RMSE=0.542854 MAE=0.343358
[Epoch 5/100] TF=1.00 | train MSE=0.031377 RMSE=0.177134 MAE=0.120206 | val MSE=0.287049 RMSE=0.535769 MAE=0.362786
[Epoch 6/100] TF=1.00 | train MSE=0.028953 RMSE=0.170156 MAE=0.114649 | val MSE=0.214834 RMSE=0.463502 MAE=0.301598
  >> Saved best model (val MSE=0.214834)
[Epoch 7/100] TF=1.00 | train MSE=0.026798 RMSE=0.163701 MAE=0.109261 | val MSE=0.195323 RMSE=0.441954 MAE=0.287313
  >> Saved best model (val MSE=0.1

##### convlstm+unet

In [ ]:
convlstm_unet_cfg = TrainConfig(
    data_dir="./",
    max_epochs=100,
    batch_size=2,
    enc_cell_type="convlstm",
    dec_cell_type="convlstm",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=True,
    unet_head=True,
    out_root="./runs_w_unet",
    viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
    viz_use_last_t=True,       # 可切换 False 以随机时间步
    viz_channel=None,          # None=随机通道；也可设 0/1/2...
    save_all_channels_png=True
)
convlstm_unet_model, convlstm_unet_history = train(convlstm_unet_cfg)


数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.214036 RMSE=0.462641 MAE=0.299690 | val MSE=0.362628 RMSE=0.602186 MAE=0.401035
  >> Saved best model (val MSE=0.362628)
[Epoch 2/100] TF=1.00 | train MSE=0.049479 RMSE=0.222439 MAE=0.152822 | val MSE=0.293458 RMSE=0.541717 MAE=0.349391
  >> Saved best model (val MSE=0.293458)
[Epoch 3/100] TF=1.00 | train MSE=0.038516 RMSE=0.196256 MAE=0.134019 | val MSE=0.279627 RMSE=0.528798 MAE=0.339137
  >> Saved best model (val MSE=0.279627)
[Epoch 4/100] TF=1.00 | train MSE=0.032509 RMSE=0.180304 MAE=0.123485 | val MSE=0.223405 RMSE=0.472657 MAE=0.315398
  >> Saved best model (val MSE=0.223405)
[Epoch 5/100] TF=1.00 | train MSE=0.029138 RMSE=0.170699 MAE=0.116285 | val MSE=0.259078 RMSE=0.508997 MAE=0.352638
[Epoch 6/100] TF=1.00 | train MSE=0.026996 RMSE=0.164303 MAE=0.111629 | val MSE=0.272162 RMSE=0.521692 MAE=0.320499
[Epoch 7/100] TF=1.00 | train MSE=0.026019 RMSE=0.161303 MAE=0.109567 | val MSE=0.229228 RMSE=0.478778 MAE=0.

#### Training

##### convgru

In [ ]:
convgru_cfg = TrainConfig(
    data_dir="./",
    max_epochs=100,
    batch_size=2,
    enc_cell_type="convgru",
    dec_cell_type="convgru",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=False,
    unet_head=False,
    out_root="./runs_wo_unet",
    viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
    viz_use_last_t=True,       # 可切换 False 以随机时间步
    viz_channel=None,          # None=随机通道；也可设 0/1/2...
    save_all_channels_png=True
)
convgru_model, convgru_history = train(convgru_cfg)


数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.222980 RMSE=0.472208 MAE=0.302870 | val MSE=0.263972 RMSE=0.513782 MAE=0.339584
  >> Saved best model (val MSE=0.263972)
[Epoch 2/100] TF=1.00 | train MSE=0.057572 RMSE=0.239941 MAE=0.157518 | val MSE=0.208743 RMSE=0.456884 MAE=0.298420
  >> Saved best model (val MSE=0.208743)
[Epoch 3/100] TF=1.00 | train MSE=0.038604 RMSE=0.196479 MAE=0.128045 | val MSE=0.192454 RMSE=0.438696 MAE=0.276414
  >> Saved best model (val MSE=0.192454)
[Epoch 4/100] TF=1.00 | train MSE=0.031092 RMSE=0.176328 MAE=0.113821 | val MSE=0.173346 RMSE=0.416349 MAE=0.257819
  >> Saved best model (val MSE=0.173346)
[Epoch 5/100] TF=1.00 | train MSE=0.027274 RMSE=0.165149 MAE=0.105648 | val MSE=0.171405 RMSE=0.414011 MAE=0.254026
  >> Saved best model (val MSE=0.171405)
[Epoch 6/100] TF=1.00 | train MSE=0.024840 RMSE=0.157607 MAE=0.100137 | val MSE=0.160515 RMSE=0.400644 MAE=0.248624
  >> Saved best model (val MSE=0.160515)
[Epoch 7/100] TF=1.00 | tra

##### convrnn

In [ ]:
convrnn_cfg = TrainConfig(
    data_dir="./",
    max_epochs=100,
    batch_size=2,
    enc_cell_type="convrnn",
    dec_cell_type="convrnn",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=False,
    unet_head=False,
    out_root="./runs_wo_unet",
    viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
    viz_use_last_t=True,       # 可切换 False 以随机时间步
    viz_channel=None,          # None=随机通道；也可设 0/1/2...
    save_all_channels_png=True
)
convrnn_model, convrnn_history = train(convrnn_cfg)


数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.228858 RMSE=0.478391 MAE=0.307553 | val MSE=0.278009 RMSE=0.527266 MAE=0.358716
  >> Saved best model (val MSE=0.278009)
[Epoch 2/100] TF=1.00 | train MSE=0.064741 RMSE=0.254443 MAE=0.168782 | val MSE=0.232402 RMSE=0.482081 MAE=0.305994
  >> Saved best model (val MSE=0.232402)
[Epoch 3/100] TF=1.00 | train MSE=0.045210 RMSE=0.212626 MAE=0.139613 | val MSE=0.218584 RMSE=0.467530 MAE=0.301286
  >> Saved best model (val MSE=0.218584)
[Epoch 4/100] TF=1.00 | train MSE=0.036992 RMSE=0.192334 MAE=0.124840 | val MSE=0.199810 RMSE=0.447001 MAE=0.282246
  >> Saved best model (val MSE=0.199810)
[Epoch 5/100] TF=1.00 | train MSE=0.032539 RMSE=0.180385 MAE=0.115509 | val MSE=0.189646 RMSE=0.435484 MAE=0.269302
  >> Saved best model (val MSE=0.189646)
[Epoch 6/100] TF=1.00 | train MSE=0.030017 RMSE=0.173254 MAE=0.109934 | val MSE=0.185114 RMSE=0.430249 MAE=0.265207
  >> Saved best model (val MSE=0.185114)
[Epoch 7/100] TF=1.00 | tra

##### convlstm

In [ ]:
convlstm_cfg = TrainConfig(
    data_dir="./",
    max_epochs=100,
    batch_size=2,
    enc_cell_type="convlstm",
    dec_cell_type="convlstm",
    n_enc_layers=1,
    n_dec_layers=1,
    enable_tf_ratio=False,
    layernorm=True,
    peephole=True,
    unet_head=False,
    out_root="./runs_wo_unet",
    viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
    viz_use_last_t=True,       # 可切换 False 以随机时间步
    viz_channel=None,          # None=随机通道；也可设 0/1/2...
    save_all_channels_png=True
)
convlstm_model, convlstm_history = train(convlstm_cfg)


数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.340915 RMSE=0.583879 MAE=0.405214 | val MSE=0.292421 RMSE=0.540759 MAE=0.365355
  >> Saved best model (val MSE=0.292421)
[Epoch 2/100] TF=1.00 | train MSE=0.106657 RMSE=0.326584 MAE=0.219064 | val MSE=0.225298 RMSE=0.474655 MAE=0.310597
  >> Saved best model (val MSE=0.225298)
[Epoch 3/100] TF=1.00 | train MSE=0.051899 RMSE=0.227814 MAE=0.151023 | val MSE=0.199800 RMSE=0.446990 MAE=0.279797
  >> Saved best model (val MSE=0.199800)
[Epoch 4/100] TF=1.00 | train MSE=0.037465 RMSE=0.193560 MAE=0.126202 | val MSE=0.186121 RMSE=0.431418 MAE=0.268888
  >> Saved best model (val MSE=0.186121)
[Epoch 5/100] TF=1.00 | train MSE=0.031833 RMSE=0.178417 MAE=0.114850 | val MSE=0.179173 RMSE=0.423288 MAE=0.259909
  >> Saved best model (val MSE=0.179173)
[Epoch 6/100] TF=1.00 | train MSE=0.028764 RMSE=0.169599 MAE=0.108098 | val MSE=0.181497 RMSE=0.426025 MAE=0.260744
[Epoch 7/100] TF=1.00 | train MSE=0.026688 RMSE=0.163364 MAE=0.10306

In [ ]:
print(model)

ConvSeq2Seq(
  (enc): RecurrentStack(
    (layers): ModuleList(
      (0): ConvGRUCell(
        (z): Conv2d(37, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (r): Conv2d(37, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (h): Conv2d(37, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (ln_z): GroupNorm(1, 32, eps=1e-05, affine=True)
        (ln_r): GroupNorm(1, 32, eps=1e-05, affine=True)
        (ln_h): GroupNorm(1, 32, eps=1e-05, affine=True)
      )
    )
  )
  (dec): RecurrentStack(
    (layers): ModuleList(
      (0): ConvGRUCell(
        (z): Conv2d(37, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (r): Conv2d(37, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (h): Conv2d(37, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (ln_z): GroupNorm(1, 32, eps=1e-05, affine=True)
        (ln_r): GroupNorm(1, 32, eps=1e-05, affine=True)
        (ln_h): GroupNorm(1, 32, eps=1e-05, affine=True)
   

In [ ]:
lstm_cfg = TrainConfig(
        data_dir="./",
        max_epochs=100,
        batch_size=2,
        enc_cell_type="convlstm",
        dec_cell_type="convlstm",
        n_enc_layers=1,
        n_dec_layers=1,
        enable_tf_ratio=False,
        layernorm=True,
        peephole=False,
        unet_head=False,
        out_root="./runs",
        viz_every_n_epochs=10,     # 每 10 个 epoch 可视化一次（train+eval）
        viz_use_last_t=True,       # 可切换 False 以随机时间步
        viz_channel=None,          # None=随机通道；也可设 0/1/2...
        save_all_channels_png=True
    )
lstm_model, lstm_history = train(lstm_cfg)

数据集总数: 119 | 训练集: 95 | 验证集: 24
[Epoch 1/100] TF=1.00 | train MSE=0.375060 | val MSE=0.310306
  >> Saved best model (val MSE=0.310306)
[Epoch 2/100] TF=1.00 | train MSE=0.119030 | val MSE=0.221961
  >> Saved best model (val MSE=0.221961)
[Epoch 3/100] TF=1.00 | train MSE=0.055772 | val MSE=0.198284
  >> Saved best model (val MSE=0.198284)
[Epoch 4/100] TF=1.00 | train MSE=0.039263 | val MSE=0.181580
  >> Saved best model (val MSE=0.181580)
[Epoch 5/100] TF=1.00 | train MSE=0.032847 | val MSE=0.168140
  >> Saved best model (val MSE=0.168140)
[Epoch 6/100] TF=1.00 | train MSE=0.029480 | val MSE=0.175374
[Epoch 7/100] TF=1.00 | train MSE=0.027318 | val MSE=0.160536
  >> Saved best model (val MSE=0.160536)
[Epoch 8/100] TF=1.00 | train MSE=0.025808 | val MSE=0.159731
  >> Saved best model (val MSE=0.159731)
[Epoch 9/100] TF=1.00 | train MSE=0.024653 | val MSE=0.148896
  >> Saved best model (val MSE=0.148896)
[Epoch 10/100] TF=1.00 | train MSE=0.023757 | val MSE=0.151932
[Epoch 11/100] TF=1.

KeyboardInterrupt: 

In [ ]:
import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

dataset = ERA5ZInOutDataset(
    data_dir="./",
    z_name="Z_2020_seq24.npy",
    meta_name="Z_2020_seq24_meta.npz",
    stats_name="Z_2020_norm_stats_2020.npz",
    normalize=True,
)

print(f"Dataset size: {len(dataset)} 条样本")
print(
    f"每条输入 patch: (T_in={dataset.input_len}, C={dataset.C}, "
    f"Hc={dataset.H}, Wc={dataset.W})"
)

# ---- 2. DataLoader ----
loader = DataLoader(
    dataset,
    batch_size=2,     # patch 很小
    shuffle=True,
    num_workers=0,    # 先 0，稳定了再开多进程！！小心这个东西，我不知道怎么选
    pin_memory=False,
)

Using device: cuda
Dataset size: 119 条样本
每条输入 patch: (T_in=16, C=5, Hc=121, Wc=240)


In [ ]:
class ConvRNNCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, layernorm: bool = False):
        super().__init__()
        p = kernel_size // 2
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.ln = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        h = torch.tanh(self.ln(self.conv(torch.cat([x, h_prev], dim=1))))
        return h

    def init_hidden(self, B, H, W, device=None, dtype=None):
        return torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)



class ConvGRUCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, layernorm: bool = False):
        super().__init__()
        p = kernel_size // 2
        self.hidden_dim = hidden_dim
        self.z = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.r = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        self.h = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=p, bias=bias)
        # 对门前激活/候选做LN（可选）
        self.ln_z = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_r = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_h = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        xh = torch.cat([x, h_prev], dim=1)
        z = torch.sigmoid(self.ln_z(self.z(xh)))
        r = torch.sigmoid(self.ln_r(self.r(xh)))
        xrh = torch.cat([x, r * h_prev], dim=1)
        h_tilde = torch.tanh(self.ln_h(self.h(xrh)))
        h = (1 - z) * h_prev + z * h_tilde
        return h

    def init_hidden(self, B, H, W, device=None, dtype=None):
        return torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)


class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, layernorm: bool = False, peephole: bool = False):
        super().__init__()
        p = kernel_size // 2
        self.hidden_dim = hidden_dim
        self.peephole = peephole

        self.conv = nn.Conv2d(input_dim + hidden_dim, 4 * hidden_dim, kernel_size, padding=p, bias=bias)
        # LN 应用于 i,f,o,g 的 pre-activations
        self.ln_i = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_f = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_o = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()
        self.ln_g = layer_norm_2d(hidden_dim) if layernorm else nn.Identity()

        if peephole:
            self.Wci = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))
            self.Wcf = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))
            self.Wco = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor, c_prev: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        gates = self.conv(torch.cat([x, h_prev], dim=1))
        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)

        # LN
        i = self.ln_i(i); f = self.ln_f(f); o = self.ln_o(o); g = self.ln_g(g)

        # Peephole on i,f (c_prev)
        if self.peephole:
            i = torch.sigmoid(i + self.Wci * c_prev)
            f = torch.sigmoid(f + self.Wcf * c_prev)
        else:
            i = torch.sigmoid(i)
            f = torch.sigmoid(f)

        g = torch.tanh(g)
        c = f * c_prev + i * g

        # Peephole on o (c_new)
        if self.peephole:
            o = torch.sigmoid(o + self.Wco * c)
        else:
            o = torch.sigmoid(o)

        h = o * torch.tanh(c)
        return h, c

    def init_hidden(self, B, H, W, device=None, dtype=None) -> Tuple[torch.Tensor, torch.Tensor]:
        h = torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)
        c = torch.zeros(B, self.hidden_dim, H, W, device=device, dtype=dtype)
        return h, c


In [ ]:
# conv_cells.py
# -*- coding: utf-8 -*-
from typing import Tuple, Optional
import torch
import torch.nn as nn


# ============================================================
# 1) 标准 ConvRNN (Elman RNN with Conv)
#    h_t = tanh( W * [x_t, h_{t-1}] )
# ============================================================
class ConvRNNCell(nn.Module):
    """
    标准卷积RNN单元（tanh 激活）。
    Args:
        input_dim: 输入通道数 C_in
        hidden_dim: 隐状态通道数 C_h
        kernel_size: 卷积核大小（奇数）
        bias: 是否使用偏置
    """
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3, bias: bool = True):
        super().__init__()
        padding = kernel_size // 2
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.conv = nn.Conv2d(input_dim + hidden_dim, hidden_dim,
                              kernel_size=kernel_size, padding=padding, bias=bias)

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        """
        x:      (B, C_in, H, W)
        h_prev: (B, C_h,  H, W)
        return: h_new (B, C_h, H, W)
        """
        combined = torch.cat([x, h_prev], dim=1)
        h_new = torch.tanh(self.conv(combined))
        return h_new

    def init_hidden(self, batch: int, height: int, width: int, device=None, dtype=None) -> torch.Tensor:
        return torch.zeros(batch, self.hidden_dim, height, width, device=device, dtype=dtype)


# ============================================================
# 2) 标准 ConvGRU
#    z_t = σ(Wz * [x, h])
#    r_t = σ(Wr * [x, h])
#    h~_t = tanh(Wh * [x, r⊙h])
#    h_t = (1 - z) ⊙ h + z ⊙ h~
# ============================================================
class ConvGRUCell(nn.Module):
    """
    标准 ConvGRU 单元。
    """
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3, bias: bool = True):
        super().__init__()
        padding = kernel_size // 2
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.conv_z = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=padding, bias=bias)
        self.conv_r = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=padding, bias=bias)
        self.conv_h = nn.Conv2d(input_dim + hidden_dim, hidden_dim, kernel_size, padding=padding, bias=bias)

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor) -> torch.Tensor:
        """
        x:      (B, C_in, H, W)
        h_prev: (B, C_h,  H, W)
        return: h_new (B, C_h, H, W)
        """
        xh = torch.cat([x, h_prev], dim=1)
        z = torch.sigmoid(self.conv_z(xh))
        r = torch.sigmoid(self.conv_r(xh))
        xrh = torch.cat([x, r * h_prev], dim=1)
        h_tilde = torch.tanh(self.conv_h(xrh))
        h_new = (1 - z) * h_prev + z * h_tilde
        return h_new

    def init_hidden(self, batch: int, height: int, width: int, device=None, dtype=None) -> torch.Tensor:
        return torch.zeros(batch, self.hidden_dim, height, width, device=device, dtype=dtype)


# ============================================================
# 3) 标准 ConvLSTM（可选 peephole）
#    i = σ(W_i * [x, h] + peephole(c_{t-1}))
#    f = σ(W_f * [x, h] + peephole(c_{t-1}))
#    g = tanh(W_g * [x, h])
#    c = f⊙c_prev + i⊙g
#    o = σ(W_o * [x, h] + peephole(c))
#    h = o⊙tanh(c)
# ============================================================
class ConvLSTMCell(nn.Module):
    """
    标准 ConvLSTM 单元（可选 peephole）。
    Args:
        input_dim: 输入通道数
        hidden_dim: 隐状态/细胞状态通道数
        kernel_size: 卷积核
        bias: 偏置
        peephole: 是否启用 peephole 连接（对 i,f,o 门引入 c 的逐元素作用）
    """
    def __init__(self, input_dim: int, hidden_dim: int, kernel_size: int = 3,
                 bias: bool = True, peephole: bool = False):
        super().__init__()
        padding = kernel_size // 2
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.peephole = peephole

        # 一次卷积得到 4 * hidden: i, f, o, g
        self.conv = nn.Conv2d(input_dim + hidden_dim, 4 * hidden_dim,
                              kernel_size=kernel_size, padding=padding, bias=bias)

        if peephole:
            # peephole 参数，逐通道、逐空间共享（C_h,1,1）
            self.Wci = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))
            self.Wcf = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))
            self.Wco = nn.Parameter(torch.zeros(1, hidden_dim, 1, 1))

    def forward(self, x: torch.Tensor, h_prev: torch.Tensor, c_prev: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        x:      (B, C_in, H, W)
        h_prev: (B, C_h,  H, W)
        c_prev: (B, C_h,  H, W)
        return: (h_new, c_new)
        """
        combined = torch.cat([x, h_prev], dim=1)
        gates = self.conv(combined)
        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)

        if self.peephole:
            # peephole: i,f 使用 c_prev；o 使用 c_new（因此先算 c_new 前要暂存）
            i = torch.sigmoid(i + self.Wci * c_prev)
            f = torch.sigmoid(f + self.Wcf * c_prev)
        else:
            i = torch.sigmoid(i)
            f = torch.sigmoid(f)

        g = torch.tanh(g)
        c_new = f * c_prev + i * g

        if self.peephole:
            o = torch.sigmoid(o + self.Wco * c_new)
        else:
            o = torch.sigmoid(o)

        h_new = o * torch.tanh(c_new)
        return h_new, c_new

    def init_hidden(self, batch: int, height: int, width: int, device=None, dtype=None) -> Tuple[torch.Tensor, torch.Tensor]:
        h = torch.zeros(batch, self.hidden_dim, height, width, device=device, dtype=dtype)
        c = torch.zeros(batch, self.hidden_dim, height, width, device=device, dtype=dtype)
        return h, c


# ============================================================
# （可选）小测试：确保能前向
# ============================================================
if __name__ == "__main__":
    B, C_in, H, W = 2, 5, 32, 64
    C_h = 16
    x = torch.randn(B, C_in, H, W)

    # ConvRNN
    rnn = ConvRNNCell(C_in, C_h, 3)
    h0 = rnn.init_hidden(B, H, W, device=x.device, dtype=x.dtype)
    h1 = rnn(x, h0)
    print("ConvRNN:", h1.shape)

    # ConvGRU
    gru = ConvGRUCell(C_in, C_h, 3)
    h0 = gru.init_hidden(B, H, W, device=x.device, dtype=x.dtype)
    h1 = gru(x, h0)
    print("ConvGRU:", h1.shape)

    # ConvLSTM (peephole off)
    lstm = ConvLSTMCell(C_in, C_h, 3, peephole=False)
    h0, c0 = lstm.init_hidden(B, H, W, device=x.device, dtype=x.dtype)
    h1, c1 = lstm(x, h0, c0)
    print("ConvLSTM (no peephole):", h1.shape, c1.shape)

    # ConvLSTM (peephole on)
    lstm_p = ConvLSTMCell(C_in, C_h, 3, peephole=True)
    h0, c0 = lstm_p.init_hidden(B, H, W, device=x.device, dtype=x.dtype)
    h1, c1 = lstm_p(x, h0, c0)
    print("ConvLSTM (peephole):", h1.shape, c1.shape)


ConvRNN: torch.Size([2, 16, 32, 64])
ConvGRU: torch.Size([2, 16, 32, 64])
ConvLSTM (no peephole): torch.Size([2, 16, 32, 64]) torch.Size([2, 16, 32, 64])
ConvLSTM (peephole): torch.Size([2, 16, 32, 64]) torch.Size([2, 16, 32, 64])
